- 外生变量只能用于作为y，进行模型拟合，而不能作为x值进行模型拟合；
- 作为x值的外生变量一定是其他模型生成的；
- 数据源中外生变量会更新，而内生变量不会；
- R-style formulas 可以自定义 函数，解决有两个【】表达式的情况；
- 定义式中还涉及其它内生变量会怎样 ？ （可能其它变量未实时更新）？
- 等式中有shift变量，是否证明AR其实意义不大？
- Eq76 是回归，有点奇怪，应该是定义式 
- 3ls如果有多个中括号，然后用简化部分的方程替代
- 转换函数一定要在中括号里面，不能在外面。等式两边都加就可以了
- R-style formula 的四则运算，乘法，除法？I()   https://zhuanlan.zhihu.com/p/51533928  
- 当多个中括号里不能同使用时，可以用I（）或定义函数
- R -style formula ,是否可以AR因素
- 方程88 ， 93 数据太少，运行出差， 每个方程加上 常量 const

In [1]:
%%capture
!pip install linearmodels

In [2]:
#引包：引入所需python包
import xlrd
import os
import re
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

from sklearn.metrics import roc_curve,auc
import matplotlib as mpl
from matplotlib import pyplot as plt
from numpy import nan
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
import seaborn as sns
import time
from collections import Counter   #引入Counter
from linearmodels import IV3SLS
from scipy.stats import kstest
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

# 数据源

In [3]:
dir = r"./data/"
print(os.listdir(dir))  #手术种类

['【20200703】-预测误差(2015-2017).xls', '【20200703】-模拟（2016，10）-3.xls', '【20200703】-模拟（2016，6-8）-2.xls', '【20200703】-模拟（2016,1-4）-1.xls', '其它数据.csv', '.ipynb_checkpoints', 'alleviewsdata.xlsx', '方程描述.csv']


In [4]:
file = "alleviewsdata.xlsx"
data_ev= pd.read_excel(dir+file,"Sheet1",index_col = 0 ,na_values=['9999'])
data_ev.columns = data_ev.columns.str.lower()
data_ev.columns

Index(['con_fn_bj', 'con_fn_bj_0', 'con_fn_bj_1', 'con_fn_bj_10',
       'con_fn_bj_2', 'con_fn_bj_3', 'con_fn_bj_4', 'con_fn_bj_5',
       'con_fn_bj_6', 'con_fn_bj_7',
       ...
       'yht_un_tj_1', 'yht_un_tj_10', 'yht_un_tj_2', 'yht_un_tj_3',
       'yht_un_tj_4', 'yht_un_tj_5', 'yht_un_tj_6', 'yht_un_tj_7',
       'yht_un_tj_8', 'yht_un_tj_9'],
      dtype='object', length=3722)

In [5]:
# 方程式的自变量，因变量
eq_disc = pd.read_csv(dir+"方程描述.csv")
eq_disc.head(2)

,ID,type,y
0,Eq1,行为,gdp1r_bj
1,Eq2,定义方程,gdpn_bj


In [6]:
len(set(list(eq_disc["y"])))

99

# 变量
共477个变量

In [7]:
list1 = ["con_fn_bj","con_fn_hb","con_fn_tj","con_fr_bj","con_fr_tj","con_gn_bj","con_gn_tj","con_gr_bj","con_gr_tj","con_pn_bj","con_pn_tj","con_pr_bj","con_pr_tj","con_prn_bj","con_prn_tj","con_prr_bj","con_prr_tj","con_pun_bj","con_pun_tj","con_pur_bj","con_pur_tj","cone_rn_bj","cone_rn_hb","cone_rn_tj","cone_un_bj","cone_un_hb","cone_un_tj","conet_rn_bj","conet_rn_hb","conet_rn_tj","conet_un_bj","conet_un_hb","conet_un_tj","consn_bj","consn_hb","consn_tj","consr_bj","consr_hb","consr_tj","cpi_bj","cpi_hb","cpi_tj","cul_area_bj","cul_area_tj","d_con_prr_tj","d_cone_rn_bj","d_cone_rn_tj","d_consn_bj","d_consn_tj","d_cpi_bj","d_cpi_tj","d_dep_san_bj","d_dep_san_hb","d_dep_san_tj","d_depn_bj","d_depn_hb","d_depn_tj","d_emp2_hb","d_emp2_tj","d_emp3_bj","d_emp3_hb","d_emp3_tj","d_exusd_bj","d_exusd_tj","d_faipi_bj","d_fen_bj_2","d_fen_hb_1","d_fen_tj_1","d_fen_tj_2","d_frn_hb","d_frn_tj","d_gdp1d_hb","d_gdp1d_tj","d_gdp1r_bj","d_gdp2_airn_tj","d_gdp2_bindar_hb","d_gdp2_bindar_tj","d_gdp2_birn_tj","d_gdp2_caln_tj","d_gdp2_emin_hb","d_gdp2_emin_tj","d_gdp2_epin_tj","d_gdp2_hemin_tj","d_gdp2_indar_bj","d_gdp2_indar_hb","d_gdp2_indar_tj","d_gdp2_mecn_tj","d_gdp2_medn_tj","d_gdp2_necin_tj","d_gdp2_nmin_tj","d_gdp2_tecn_hb_1","d_gdp2_tecn_hb_2","d_gdp2_tecn_tj_1","d_gdp2_tecn_tj_2","d_gdp2_tran_tj","d_gdp2d_bj","d_gdp2d_hb","d_gdp2d_tj","d_gdp2r_bj_1","d_gdp2r_hb_1","d_gdp2r_tj_1","d_gdp3_fian_hb","d_gdp3_infn_hb","d_gdp3_infwn_tj","d_gdp3_rean_bj","d_gdp3_renn_hb","d_gdp3_renn_tj","d_gdp3_sicn_hb","d_gdp3_sicn_tj","d_gdp3_tspan_tj","d_gdp3_wran_hb","d_gdp3_wran_tj","d_gdp3r_hb","d_gdp3r_tj","d_gdp_infn_bj","d_gdp_mosn_bj","d_gdp_tecn_bj","d_gdp_tran_bj","d_imusd_bj","d_imusd_hb","d_imusd_tj","d_investr_bj","d_investr_tj","d_loann_bj","d_nex_gsr_bj","d_nex_gsr_tj","d_popu_bj","d_ppi_bj","d_ppi_hb","d_ppi_tj","d_rpi_bj","d_tax_cinn_hb","d_tax_pinn_hb","d_tax_pinn_tj","d_tax_vadn_bj","d_tax_vadn_hb","d_tax_vadn_tj","d_taxn_bj_1","d_taxn_hb_1","d_taxn_hb_2","d_taxn_tj_1","d_taxn_tj_2","d_totemp_bj","d_totpop_bj","d_totpop_tj","d_tta_bj","d_tta_hb","d_tta_tj","d_yht_un_hb","dep_san_bj","dep_san_hb","dep_san_tj","depn_bj","depn_hb","depn_tj","emp1_bj","emp1_hb","emp1_tj","emp2_bj","emp2_bj_a","emp2_hb","emp2_tj","emp3_bj","emp3_hb","emp3_tj","exrate_cn","exusd_bj","exusd_ch","exusd_hb","exusd_tj","faipi_bj","faipi_hb","faipi_tj","fe_culn_bj","fe_culn_hb","fe_culn_tj","fe_edun_bj","fe_edun_hb","fe_edun_tj","fe_eepn_bj","fe_eepn_hb","fe_eepn_tj","fe_medn_bj","fe_medn_hb","fe_medn_tj","fe_ssen_bj","fe_ssen_hb","fe_ssen_tj","fe_tecn_bj","fe_tecn_hb","fe_tecn_tj","fen_bj","fen_bj1","fen_bj2","fen_hb","fen_hb1","fen_hb2","fen_tj","fen_tj1","fen_tj2","frn_bj","frn_hb","frn_tj","gdp1d_bj","gdp1d_hb","gdp1d_tj","gdp1n_bj","gdp1n_hb","gdp1n_tj","gdp1r_bj","gdp1r_hb","gdp1r_tj","gdp2_airn_tj","gdp2_bindad_bj","gdp2_bindad_hb","gdp2_bindad_tj","gdp2_bindan_bj","gdp2_bindan_hb","gdp2_bindan_tj","gdp2_bindar_bj","gdp2_bindar_hb","gdp2_bindar_tj","gdp2_birn_tj","gdp2_caln_tj","gdp2_emin_hb","gdp2_emin_tj","gdp2_emin_tj_a","gdp2_emir_hb","gdp2_epin_tj","gdp2_hemin_tj","gdp2_hemin_tj_a","gdp2_indad_bj","gdp2_indad_hb","gdp2_indad_tj","gdp2_indan_bj","gdp2_indan_hb","gdp2_indan_tj","gdp2_indar_bj","gdp2_indar_hb","gdp2_indar_tj","gdp2_mecn_tj","gdp2_medn_tj","gdp2_medn_tj_a","gdp2_necin_tj","gdp2_necin_tj_a","gdp2_nein_tj","gdp2_nmin_tj","gdp2_tecn_hb","gdp2_tecn_hb1","gdp2_tecn_hb2","gdp2_tecn_tj","gdp2_tecn_tj1","gdp2_tecn_tj2","gdp2_tecr_hb","gdp2_tran_tj","gdp2d_bj","gdp2d_hb","gdp2d_tj","gdp2n_bj","gdp2n_hb","gdp2n_tj","gdp2r_bj","gdp2r_bj1","gdp2r_bj2","gdp2r_erro_bj","gdp2r_erro_hb","gdp2r_erro_tj","gdp2r_hb","gdp2r_hb1","gdp2r_hb2","gdp2r_tj","gdp2r_tj1","gdp2r_tj2","gdp3_fian_bj","gdp3_fian_hb","gdp3_fian_tj","gdp3_fiar_bj","gdp3_fiar_tj","gdp3_infn_bj","gdp3_infn_hb","gdp3_infn_tj","gdp3_infr_bj","gdp3_infr_tj","gdp3_infwn_tj","gdp3_rean_bj","gdp3_rean_hb","gdp3_rean_tj","gdp3_rear_bj","gdp3_rear_tj","gdp3_renn_bj","gdp3_renn_hb","gdp3_renn_tj","gdp3_renr_bj","gdp3_renr_tj","gdp3_sicn_bj","gdp3_sicn_hb","gdp3_sicn_tj","gdp3_sicr_bj","gdp3_sicr_tj","gdp3_tragt_bj","gdp3_tragt_hb","gdp3_trapt_bj","gdp3_trapt_hb","gdp3_trapt_tj","gdp3_tratp_bj","gdp3_tratp_tj","gdp3_tspan_bj","gdp3_tspan_hb","gdp3_tspan_tj","gdp3_tspar_bj","gdp3_tspar_tj","gdp3_wran_bj","gdp3_wran_hb","gdp3_wran_tj","gdp3_wrar_bj","gdp3_wrar_tj","gdp3d_bj","gdp3d_hb","gdp3d_tj","gdp3d_tj_a","gdp3n_bj","gdp3n_hb","gdp3n_tj","gdp3r_bj","gdp3r_hb","gdp3r_tj","gdp_infn_bj","gdp_momn_bj","gdp_mosn_bj","gdp_tecn_bj","gdp_tran_bj","gdpd_bj","gdpd_hb","gdpd_tj","gdpen_bj","gdpen_hb","gdpen_tj","gdper_bj","gdper_hb","gdper_tj","gdpn_bj","gdpn_hb","gdpn_tj","gdpr_bj","gdpr_hb","gdpr_tj","imusd_bj","imusd_hb","imusd_tj","intratel_cn","inv_chn_bj","inv_chn_tj","inv_chr_bj","inv_chr_tj","inv_fin_bj","inv_fin_tj","inv_fir_bj","inv_fir_tj","inv_fon_bj","inv_fon_hb","inv_fon_tj","inv_for_bj","inv_for_tj","inv_rean_bj","inv_rean_hb","inv_rean_tj","investn_bj","investn_hb","investn_tj","investr_bj","investr_hb","investr_tj","loan_ln_bj","loan_ln_hb","loan_ln_tj","loann_bj","loann_hb","loann_tj","lse_bj","lse_hb","lse_tj","nex_gsn_bj","nex_gsn_hb","nex_gsn_tj","nex_gsr_bj","nex_gsr_bj_a","nex_gsr_tj","nex_gsr_tj_a","pcon_gd_bj","pcon_gd_tj","pcon_prd_bj","pcon_prd_tj","pcon_pud_bj","pcon_pud_tj","pgdp2_emin_hb","pgdp2_tecd_hb","pgdp3_fiad_bj","pgdp3_fiad_tj","pgdp3_infd_bj","pgdp3_infd_tj","pgdp3_read_bj","pgdp3_read_tj","pgdp3_rend_bj","pgdp3_rend_tj","pgdp3_sicd_bj","pgdp3_sicd_tj","pgdp3_tspad_bj","pgdp3_tspad_tj","pgdp3_wrad_bj","pgdp3_wrad_tj","pinv_chd_bj","pinv_chd_tj","pinv_fid_bj","pinv_fid_tj","pinv_fod_bj","pinv_fod_tj","pnex_gsd_bj","pnex_gsd_tj","popr_bj","popr_hb","popr_tj","popr_tj_a","popu_bj","popu_hb","popu_tj","ppi_bj","ppi_hb","ppi_tj","rpi_bj","rpi_hb","rpi_tj","tax_cinn_bj","tax_cinn_hb","tax_cinn_tj","tax_pinn_bj","tax_pinn_hb","tax_pinn_tj","tax_vadn_bj","tax_vadn_hb","tax_vadn_tj","taxn_bj","taxn_bj1","taxn_bj2","taxn_hb","taxn_hb1","taxn_hb2","taxn_tj","taxn_tj1","taxn_tj2","totemp_bj","totemp_hb","totemp_tj","totpop_bj","totpop_hb","totpop_tj","tta_bj","tta_f_bj","tta_hb","tta_tj","yh_rn_bj","yh_rn_hb","yh_rn_tj","yh_un_bj","yh_un_hb","yh_un_tj","yhn_bj","yhn_hb","yhn_tj","yht_n_bj","yht_n_hb","yht_n_tj","yht_rn_bj","yht_rn_hb","yht_rn_tj","yht_un_bj","yht_un_hb","yht_un_tj"]

In [8]:
allvariables = ["con_fn_bj","con_fn_hb","con_fn_tj","con_fr_bj","con_fr_tj","con_gn_bj","con_gn_tj","con_gr_bj","con_gr_tj","con_pn_bj","con_pn_tj","con_pr_bj","con_pr_tj","con_prn_bj","con_prn_tj","con_prr_bj","con_prr_tj","con_pun_bj","con_pun_tj","con_pur_bj","con_pur_tj","cone_rn_bj","cone_rn_hb","cone_rn_tj","cone_un_bj","cone_un_hb","cone_un_tj","conet_rn_bj","conet_rn_hb","conet_rn_tj","conet_un_bj","conet_un_hb","conet_un_tj","consn_bj","consn_hb","consn_tj","consr_bj","consr_hb","consr_tj","cpi_bj","cpi_hb","cpi_tj","cul_area_bj","cul_area_tj","d_con_prr_tj","d_cone_rn_bj","d_cone_rn_tj","d_consn_bj","d_consn_tj","d_cpi_bj","d_cpi_tj","d_dep_san_bj","d_dep_san_hb","d_dep_san_tj","d_depn_bj","d_depn_hb","d_depn_tj","d_emp2_hb","d_emp2_tj","d_emp3_bj","d_emp3_hb","d_emp3_tj","d_exusd_bj","d_exusd_tj","d_faipi_bj","d_fen_bj2","d_fen_hb_1","d_fen_tj1","d_fen_tj2","d_frn_hb","d_frn_tj","d_gdp1d_hb","d_gdp1d_tj","d_gdp1r_bj","d_gdp2_airn_tj","d_gdp2_bindar_hb","d_gdp2_bindar_tj","d_gdp2_birn_tj","d_gdp2_caln_tj","d_gdp2_emin_hb","d_gdp2_emin_tj","d_gdp2_epin_tj","d_gdp2_hemin_tj","d_gdp2_indar_bj","d_gdp2_indar_hb","d_gdp2_indar_tj","d_gdp2_mecn_tj","d_gdp2_medn_tj","d_gdp2_necin_tj","d_gdp2_nmin_tj","d_gdp2_tecn_hb_1","d_gdp2_tecn_hb_2","d_gdp2_tecn_tj_1","d_gdp2_tecn_tj_2","d_gdp2_tran_tj","d_gdp2d_bj","d_gdp2d_hb","d_gdp2d_tj","d_gdp2r_bj_1","d_gdp2r_hb_1","d_gdp2r_tj_1","d_gdp3_fian_hb","d_gdp3_infn_hb","d_gdp3_infwn_tj","d_gdp3_rean_bj","d_gdp3_renn_hb","d_gdp3_renn_tj","d_gdp3_sicn_hb","d_gdp3_sicn_tj","d_gdp3_tspan_tj","d_gdp3_wran_hb","d_gdp3_wran_tj","d_gdp3r_hb","d_gdp3r_tj","d_gdp_infn_bj","d_gdp_mosn_bj","d_gdp_tecn_bj","d_gdp_tran_bj","d_imusd_bj","d_imusd_hb","d_imusd_tj","d_investr_bj","d_investr_tj","d_loann_bj","d_nex_gsr_bj","d_nex_gsr_tj","d_popu_bj","d_ppi_bj","d_ppi_hb","d_ppi_tj","d_rpi_bj","d_tax_cinn_hb","d_tax_pinn_hb","d_tax_pinn_tj","d_tax_vadn_bj","d_tax_vadn_hb","d_tax_vadn_tj","d_taxn_bj_1","d_taxn_hb_1","d_taxn_hb_2","d_taxn_tj_1","d_taxn_tj_2","d_totemp_bj","d_totpop_bj","d_totpop_tj","d_tta_bj","d_tta_hb","d_tta_tj","d_yht_un_hb","dep_san_bj","dep_san_hb","dep_san_tj","depn_bj","depn_hb","depn_tj","emp1_bj","emp1_hb","emp1_tj","emp2_bj","emp2_bj_a","emp2_hb","emp2_tj","emp3_bj","emp3_hb","emp3_tj","exrate_cn","exusd_bj","exusd_ch","exusd_hb","exusd_tj","faipi_bj","faipi_hb","faipi_tj","fe_culn_bj","fe_culn_hb","fe_culn_tj","fe_edun_bj","fe_edun_hb","fe_edun_tj","fe_eepn_bj","fe_eepn_hb","fe_eepn_tj","fe_medn_bj","fe_medn_hb","fe_medn_tj","fe_ssen_bj","fe_ssen_hb","fe_ssen_tj","fe_tecn_bj","fe_tecn_hb","fe_tecn_tj","fen_bj","fen_bj1","fen_bj2","fen_hb","fen_hb1","fen_hb2","fen_tj","fen_tj1","fen_tj2","frn_bj","frn_hb","frn_tj","gdp1d_bj","gdp1d_hb","gdp1d_tj","gdp1n_bj","gdp1n_hb","gdp1n_tj","gdp1r_bj","gdp1r_hb","gdp1r_tj","gdp2_airn_tj","gdp2_bindad_bj","gdp2_bindad_hb","gdp2_bindad_tj","gdp2_bindan_bj","gdp2_bindan_hb","gdp2_bindan_tj","gdp2_bindar_bj","gdp2_bindar_hb","gdp2_bindar_tj","gdp2_birn_tj","gdp2_caln_tj","gdp2_emin_hb","gdp2_emin_tj","gdp2_emin_tj_a","gdp2_emir_hb","gdp2_epin_tj","gdp2_hemin_tj","gdp2_hemin_tj_a","gdp2_indad_bj","gdp2_indad_hb","gdp2_indad_tj","gdp2_indan_bj","gdp2_indan_hb","gdp2_indan_tj","gdp2_indar_bj","gdp2_indar_hb","gdp2_indar_tj","gdp2_mecn_tj","gdp2_medn_tj","gdp2_medn_tj_a","gdp2_necin_tj","gdp2_necin_tj_a","gdp2_nein_tj","gdp2_nmin_tj","gdp2_tecn_hb","gdp2_tecn_hb1","gdp2_tecn_hb2","gdp2_tecn_tj","gdp2_tecn_tj1","gdp2_tecn_tj2","gdp2_tecr_hb","gdp2_tran_tj","gdp2d_bj","gdp2d_hb","gdp2d_tj","gdp2n_bj","gdp2n_hb","gdp2n_tj","gdp2r_bj","gdp2r_bj1","gdp2r_bj2","gdp2r_erro_bj","gdp2r_erro_hb","gdp2r_erro_tj","gdp2r_hb","gdp2r_hb1","gdp2r_hb2","gdp2r_tj","gdp2r_tj1","gdp2r_tj2","gdp3_fian_bj","gdp3_fian_hb","gdp3_fian_tj","gdp3_fiar_bj","gdp3_fiar_tj","gdp3_infn_bj","gdp3_infn_hb","gdp3_infn_tj","gdp3_infr_bj","gdp3_infr_tj","gdp3_infwn_tj","gdp3_rean_bj","gdp3_rean_hb","gdp3_rean_tj","gdp3_rear_bj","gdp3_rear_tj","gdp3_renn_bj","gdp3_renn_hb","gdp3_renn_tj","gdp3_renr_bj","gdp3_renr_tj","gdp3_sicn_bj","gdp3_sicn_hb","gdp3_sicn_tj","gdp3_sicr_bj","gdp3_sicr_tj","gdp3_tragt_bj","gdp3_tragt_hb","gdp3_trapt_bj","gdp3_trapt_hb","gdp3_trapt_tj","gdp3_tratp_bj","gdp3_tratp_tj","gdp3_tspan_bj","gdp3_tspan_hb","gdp3_tspan_tj","gdp3_tspar_bj","gdp3_tspar_tj","gdp3_wran_bj","gdp3_wran_hb","gdp3_wran_tj","gdp3_wrar_bj","gdp3_wrar_tj","gdp3d_bj","gdp3d_hb","gdp3d_tj","gdp3d_tj_a","gdp3n_bj","gdp3n_hb","gdp3n_tj","gdp3r_bj","gdp3r_hb","gdp3r_tj","gdp_infn_bj","gdp_momn_bj","gdp_mosn_bj","gdp_tecn_bj","gdp_tran_bj","gdpd_bj","gdpd_hb","gdpd_tj","gdpen_bj","gdpen_hb","gdpen_tj","gdper_bj","gdper_hb","gdper_tj","gdpn_bj","gdpn_hb","gdpn_tj","gdpr_bj","gdpr_hb","gdpr_tj","imusd_bj","imusd_hb","imusd_tj","intratel_cn","inv_chn_bj","inv_chn_tj","inv_chr_bj","inv_chr_tj","inv_fin_bj","inv_fin_tj","inv_fir_bj","inv_fir_tj","inv_fon_bj","inv_fon_hb","inv_fon_tj","inv_for_bj","inv_for_tj","inv_rean_bj","inv_rean_hb","inv_rean_tj","investn_bj","investn_hb","investn_tj","investr_bj","investr_hb","investr_tj","loan_ln_bj","loan_ln_hb","loan_ln_tj","loann_bj","loann_hb","loann_tj","lse_bj","lse_hb","lse_tj","nex_gsn_bj","nex_gsn_hb","nex_gsn_tj","nex_gsr_bj","nex_gsr_bj_a","nex_gsr_tj","nex_gsr_tj_a","pcon_gd_bj","pcon_gd_tj","pcon_prd_bj","pcon_prd_tj","pcon_pud_bj","pcon_pud_tj","pgdp2_emin_hb","pgdp2_tecd_hb","pgdp3_fiad_bj","pgdp3_fiad_tj","pgdp3_infd_bj","pgdp3_infd_tj","pgdp3_read_bj","pgdp3_read_tj","pgdp3_rend_bj","pgdp3_rend_tj","pgdp3_sicd_bj","pgdp3_sicd_tj","pgdp3_tspad_bj","pgdp3_tspad_tj","pgdp3_wrad_bj","pgdp3_wrad_tj","pinv_chd_bj","pinv_chd_tj","pinv_fid_bj","pinv_fid_tj","pinv_fod_bj","pinv_fod_tj","pnex_gsd_bj","pnex_gsd_tj","popr_bj","popr_hb","popr_tj","popr_tj_a","popu_bj","popu_hb","popu_tj","ppi_bj","ppi_hb","ppi_tj","rpi_bj","rpi_hb","rpi_tj","tax_cinn_bj","tax_cinn_hb","tax_cinn_tj","tax_pinn_bj","tax_pinn_hb","tax_pinn_tj","tax_vadn_bj","tax_vadn_hb","tax_vadn_tj","taxn_bj","taxn_bj1","taxn_bj2","taxn_hb","taxn_hb1","taxn_hb2","taxn_tj","taxn_tj1","taxn_tj2","totemp_bj","totemp_hb","totemp_tj","totpop_bj","totpop_hb","totpop_tj","tta_bj","tta_f_bj","tta_hb","tta_tj","yh_rn_bj","yh_rn_hb","yh_rn_tj","yh_un_bj","yh_un_hb","yh_un_tj","yhn_bj","yhn_hb","yhn_tj","yht_n_bj","yht_n_hb","yht_n_tj","yht_rn_bj","yht_rn_hb","yht_rn_tj"]
allvariables = [x.lower() for x in allvariables]
allvariables = [x for x in allvariables if x != ""]
len(allvariables)

474

## 284个内生变量
- 模型运行后都有一个估值 ：*_0

- 估值又会作为外生变量进去其他模型
- eviews的内生变量是以方程的名字命名，而不一定完全等于 series的名字。但python需要变量名。
- 以"*_数值”结束的变量，都要改掉

In [9]:
exogs_all = ["GDP1R_BJ","GDPn_bj","GDPr_bj","GDP1n_bj","GDP2_indaN_bj","gdp2_bindan_bj","GDP2N_bj","GDP2R_bj2","gdp2r_bj1","gdp2r_bj","GDP3N_bj","GDP2_INDAR_BJ","GDP2_BINDAR_BJ","GDP3R_BJ","con_fR_bj","con_gR_bj","con_pR_bj","con_fN_bj","con_pN_bj","con_puN_bj","con_prN_bj","con_gn_bj","investN_bj","inv_foN_bj","inv_chN_bj","consR_bj","inv_foR_bj","inv_fiN_bj","GDPeR_bj","GDPeN_bj","nex_gsN_bj","CON_PUR_BJ","CON_PRR_BJ","INVESTR_BJ","INV_FIR_BJ","NEX_GSR_BJ","CONSN_BJ","EXUSD_BJ","IMUSD_BJ","yht_N_bj","yh_uN_bj","yh_rN_bj","yhN_bj","conet_uN_bj","conet_rN_bj","YHT_UN_BJ","YHT_RN_BJ","CONE_UN_BJ","CONE_RN_BJ","FRN_BJ","TAXN_BJ_1","TAXN_BJ_2","taxn_bj","TAX_VADN_BJ","TAX_CINN_BJ","TAX_PINN_BJ","FEN_BJ_1","FEN_BJ_2","feN_bj","DEPN_BJ","DEP_SAN_BJ","LOAN_LN_BJ","LOANN_BJ","emp1_bj","POPR_bj","TOTEMP_BJ","EMP2_BJ","","EMP3_BJ","TOTPOP_BJ","POPU_BJ","RPI_BJ","CPI_BJ","GDP1D_BJ","GDP2D_BJ","GDP3D_BJ","GDPD_BJ","PPI_BJ","FAIPI_BJ","gdp3_tspaR_bj","GDP3_TSPAN_BJ","gdp3_wrar_bj","GDP3_WRAN_BJ","gdp3_fiar_bj","GDP3_FIAN_BJ","gdp3_rear_bj","GDP3_REAN_BJ","gdp3_infr_bj","GDP3_INFN_BJ","gdp3_renr_bj","GDP3_RENN_BJ","gdp3_sicr_bj","GDP3_SICN_BJ","GDP_TRAN_BJ","GDP_TECN_BJ","TTA_F_BJ","GDP_MOSN_BJ","gdp_momn_bj","GDP_INFN_BJ","tta_bj","GDP1R_TJ","GDPn_tj","GDPr_tj","GDP1n_tj","GDP2_indaN_tj","gdp2_bindan_tj","GDP2N_tj","GDP2R_tj2","gdp2r_tj_1","gdp2r_tj","GDP3N_tj","GDP2_INDAR_TJ","GDP2_BINDAR_TJ","GDP3R_TJ","con_fR_tj","con_gR_tj","con_pR_tj","con_fN_tj","con_pN_tj","con_puN_tj","con_prN_tj","con_gn_tj","investN_tj","inv_foN_tj","inv_chN_tj","consR_tj","inv_foR_tj","inv_fiN_tj","GDPeR_tj","GDPeN_tj","nex_gsN_tj","CON_PUR_TJ","CON_PRR_TJ","INVESTR_TJ","INV_FIR_TJ","NEX_GSR_TJ","","CONSN_TJ","EXUSD_TJ","IMUSD_TJ","yht_N_tj","yh_uN_tj","yh_rN_tj","yhN_tj","conet_uN_tj","conet_rN_tj","YHT_UN_TJ","YHT_RN_TJ","CONE_UN_TJ","CONE_RN_TJ","FRN_TJ","TAXN_TJ_1","TAXN_TJ_2","taxn_tj","TAX_VADN_TJ","TAX_CINN_TJ","TAX_PINN_TJ","FEN_TJ_1","FEN_TJ_2","feN_tj","DEPN_TJ","DEP_SAN_TJ","LOAN_LN_TJ","LOANN_TJ","emp1_tj","POPR_tj","","TOTEMP_TJ","EMP2_TJ","EMP3_TJ","TOTPOP_TJ","POPU_TJ","RPI_TJ","CPI_TJ","GDP1D_TJ","GDP2D_TJ","GDP3D_TJ","","GDPD_TJ","PPI_TJ","FAIPI_TJ","gdp3_tspaR_tj","GDP3_TSPAN_TJ","gdp3_wrar_tj","GDP3_WRAN_TJ","gdp3_fiar_tj","GDP3_FIAN_TJ","gdp3_rear_tj","GDP3_REAN_TJ","gdp3_infr_tj","GDP3_INFN_TJ","gdp3_renr_tj","GDP3_RENN_TJ","gdp3_sicr_tj","GDP3_SICN_TJ","GDP2_TECN_TJ_1","GDP2_TECN_TJ_2","GDP2_TECN_TJ","GDP2_MEDN_TJ","","TTA_TJ","GDP2_AIRN_TJ","GDP2_CALN_TJ","GDP2_MECN_TJ","GDP2_EMIN_TJ","GDP2_TRAN_TJ","GDP3_INFWN_TJ","GDP2_HEMIN_TJ","","GDP2_NMIN_TJ","GDP2_BIRN_TJ","GDP2_NECIN_TJ","","GDP2_NEIN_TJ","GDP2_EPIN_TJ","GDP1R_HB","GDPn_hb","GDPr_hb","GDP1n_hb","GDP2_indaN_hb","gdp2_bindan_hb","GDP2N_hb","GDP2R_hb2","gdp2r_hb_1","gdp2r_hb","GDP3N_hb","GDP2_INDAR_HB","GDP2_BINDAR_HB","GDP3R_HB","investN_hb","consR_hb","GDPeR_hb","GDPeN_hb","con_fn_hb","INVESTR_HB","CONSN_HB","EXUSD_HB","IMUSD_HB","yht_N_hb","yh_uN_hb","yh_rN_hb","yhN_hb","conet_uN_hb","conet_rN_hb","YHT_UN_HB","YHT_RN_HB","CONE_UN_HB","CONE_RN_HB","FRN_HB","TAXN_HB_1","TAXN_HB_2","taxn_hb","TAX_VADN_HB","TAX_CINN_HB","TAX_PINN_HB","FEN_HB_1","FEN_HB_2","feN_hb","DEPN_HB","DEP_SAN_HB","LOAN_LN_HB","LOANN_HB","emp1_hb","POPR_hb","TOTEMP_HB","EMP2_HB","EMP3_HB","TOTPOP_HB","POPU_HB","RPI_HB","CPI_HB","GDP1D_HB","GDP2D_HB","GDP3D_HB","GDPD_HB","PPI_HB","FAIPI_HB","GDP3_TSPAN_HB","GDP3_WRAN_HB","GDP3_FIAN_HB","GDP3_REAN_HB","GDP3_INFN_HB","GDP3_RENN_HB","GDP3_SICN_HB","GDP2_TECN_HB_1","GDP2_TECN_HB_2","gdp2_tecn_hb","TTA_HB","gdp2_tecR_hb","GDP2_EMIN_HB","gdp2_emiR_hb"]
exogs_all = [x.lower() for x in exogs_all]
exogs_all = [x for x in exogs_all if x != ""]
len(exogs_all)

284

In [10]:
exogs_all = [x.replace("bj_1","bj1") for x in exogs_all  ]
exogs_all = [x.replace("bj_2","bj2") for x in exogs_all  ]
exogs_all = [x.replace("hb_1","hb1") for x in exogs_all  ]
exogs_all = [x.replace("hb_2","hb2") for x in exogs_all  ]
exogs_all = [x.replace("tj_1","tj1") for x in exogs_all  ]
exogs_all = [x.replace("tj_2","tj2") for x in exogs_all  ]
exogs_all

['gdp1r_bj',
 'gdpn_bj',
 'gdpr_bj',
 'gdp1n_bj',
 'gdp2_indan_bj',
 'gdp2_bindan_bj',
 'gdp2n_bj',
 'gdp2r_bj2',
 'gdp2r_bj1',
 'gdp2r_bj',
 'gdp3n_bj',
 'gdp2_indar_bj',
 'gdp2_bindar_bj',
 'gdp3r_bj',
 'con_fr_bj',
 'con_gr_bj',
 'con_pr_bj',
 'con_fn_bj',
 'con_pn_bj',
 'con_pun_bj',
 'con_prn_bj',
 'con_gn_bj',
 'investn_bj',
 'inv_fon_bj',
 'inv_chn_bj',
 'consr_bj',
 'inv_for_bj',
 'inv_fin_bj',
 'gdper_bj',
 'gdpen_bj',
 'nex_gsn_bj',
 'con_pur_bj',
 'con_prr_bj',
 'investr_bj',
 'inv_fir_bj',
 'nex_gsr_bj',
 'consn_bj',
 'exusd_bj',
 'imusd_bj',
 'yht_n_bj',
 'yh_un_bj',
 'yh_rn_bj',
 'yhn_bj',
 'conet_un_bj',
 'conet_rn_bj',
 'yht_un_bj',
 'yht_rn_bj',
 'cone_un_bj',
 'cone_rn_bj',
 'frn_bj',
 'taxn_bj1',
 'taxn_bj2',
 'taxn_bj',
 'tax_vadn_bj',
 'tax_cinn_bj',
 'tax_pinn_bj',
 'fen_bj1',
 'fen_bj2',
 'fen_bj',
 'depn_bj',
 'dep_san_bj',
 'loan_ln_bj',
 'loann_bj',
 'emp1_bj',
 'popr_bj',
 'totemp_bj',
 'emp2_bj',
 'emp3_bj',
 'totpop_bj',
 'popu_bj',
 'rpi_bj',
 'cpi_bj',
 '

In [15]:
# 外生变量清单和方程式的y值对应上了
[x for x in list(eq_disc["y"]) if x not in exogs_all]

[]

<font  color="blue" size =4>没有重复的内生变量 </font>

In [16]:
b = dict(Counter(exogs_all))
print ({key:value for key,value in b.items()if value > 1})  #展现重复元素和重复次数 

{}


<font  color="blue" size =4>各地区数量正确： </font>

In [17]:
print ("北京的内生变量数量 ok ： " , len([x for x in exogs_all if "bj" in  x]))
print ("天津的内生变量数量 ： " , len([x for x in exogs_all if "tj" in  x]))
print ("河北的内生变量数量 ： " , len([x for x in exogs_all if "hb" in  x]))

北京的内生变量数量 ok ：  99
天津的内生变量数量 ：  109
河北的内生变量数量 ：  76


## 184个外生变量

In [11]:
endog = [ x for x in list1 if x not in exogs_all]
len(endog)

193

In [12]:
endog

['cul_area_bj',
 'cul_area_tj',
 'd_con_prr_tj',
 'd_cone_rn_bj',
 'd_cone_rn_tj',
 'd_consn_bj',
 'd_consn_tj',
 'd_cpi_bj',
 'd_cpi_tj',
 'd_dep_san_bj',
 'd_dep_san_hb',
 'd_dep_san_tj',
 'd_depn_bj',
 'd_depn_hb',
 'd_depn_tj',
 'd_emp2_hb',
 'd_emp2_tj',
 'd_emp3_bj',
 'd_emp3_hb',
 'd_emp3_tj',
 'd_exusd_bj',
 'd_exusd_tj',
 'd_faipi_bj',
 'd_fen_bj_2',
 'd_fen_hb_1',
 'd_fen_tj_1',
 'd_fen_tj_2',
 'd_frn_hb',
 'd_frn_tj',
 'd_gdp1d_hb',
 'd_gdp1d_tj',
 'd_gdp1r_bj',
 'd_gdp2_airn_tj',
 'd_gdp2_bindar_hb',
 'd_gdp2_bindar_tj',
 'd_gdp2_birn_tj',
 'd_gdp2_caln_tj',
 'd_gdp2_emin_hb',
 'd_gdp2_emin_tj',
 'd_gdp2_epin_tj',
 'd_gdp2_hemin_tj',
 'd_gdp2_indar_bj',
 'd_gdp2_indar_hb',
 'd_gdp2_indar_tj',
 'd_gdp2_mecn_tj',
 'd_gdp2_medn_tj',
 'd_gdp2_necin_tj',
 'd_gdp2_nmin_tj',
 'd_gdp2_tecn_hb_1',
 'd_gdp2_tecn_hb_2',
 'd_gdp2_tecn_tj_1',
 'd_gdp2_tecn_tj_2',
 'd_gdp2_tran_tj',
 'd_gdp2d_bj',
 'd_gdp2d_hb',
 'd_gdp2d_tj',
 'd_gdp2r_bj_1',
 'd_gdp2r_hb_1',
 'd_gdp2r_tj_1',
 'd_gdp3_f

## 9个因子

In [4]:
Eq36 加入的因子 @ADD NEX_GSR_BJ  NEX_GSR_BJ_A

SyntaxError: invalid syntax (<ipython-input-4-acd057dde369>, line 1)

In [ ]:
Eq 67 @ADD EMP2_BJ  EMP2_BJ_A
Eq 135 @ADD NEX_GSR_TJ  NEX_GSR_TJ_A
Eq 164 @ADD POPR_TJ  POPR_TJ_A
Eq 174 @ADD GDP3D_TJ  GDP3D_TJ_A
Eq 195 @ADD GDP2_MEDN_TJ  GDP2_MEDN_TJ_A
Eq 200 @ADD GDP2_EMIN_TJ  GDP2_EMIN_TJ_A
Eq 203 @ADD GDP2_HEMIN_TJ  GDP2_HEMIN_TJ_A
Eq 206 @ADD GDP2_NECIN_TJ  GDP2_NECIN_TJ_A

# 方程
## 方程ID

In [13]:
Eqlist = ["Eq"+str(i+1) for i in range(284)]
Eqlist

['Eq1',
 'Eq2',
 'Eq3',
 'Eq4',
 'Eq5',
 'Eq6',
 'Eq7',
 'Eq8',
 'Eq9',
 'Eq10',
 'Eq11',
 'Eq12',
 'Eq13',
 'Eq14',
 'Eq15',
 'Eq16',
 'Eq17',
 'Eq18',
 'Eq19',
 'Eq20',
 'Eq21',
 'Eq22',
 'Eq23',
 'Eq24',
 'Eq25',
 'Eq26',
 'Eq27',
 'Eq28',
 'Eq29',
 'Eq30',
 'Eq31',
 'Eq32',
 'Eq33',
 'Eq34',
 'Eq35',
 'Eq36',
 'Eq37',
 'Eq38',
 'Eq39',
 'Eq40',
 'Eq41',
 'Eq42',
 'Eq43',
 'Eq44',
 'Eq45',
 'Eq46',
 'Eq47',
 'Eq48',
 'Eq49',
 'Eq50',
 'Eq51',
 'Eq52',
 'Eq53',
 'Eq54',
 'Eq55',
 'Eq56',
 'Eq57',
 'Eq58',
 'Eq59',
 'Eq60',
 'Eq61',
 'Eq62',
 'Eq63',
 'Eq64',
 'Eq65',
 'Eq66',
 'Eq67',
 'Eq68',
 'Eq69',
 'Eq70',
 'Eq71',
 'Eq72',
 'Eq73',
 'Eq74',
 'Eq75',
 'Eq76',
 'Eq77',
 'Eq78',
 'Eq79',
 'Eq80',
 'Eq81',
 'Eq82',
 'Eq83',
 'Eq84',
 'Eq85',
 'Eq86',
 'Eq87',
 'Eq88',
 'Eq89',
 'Eq90',
 'Eq91',
 'Eq92',
 'Eq93',
 'Eq94',
 'Eq95',
 'Eq96',
 'Eq97',
 'Eq98',
 'Eq99',
 'Eq100',
 'Eq101',
 'Eq102',
 'Eq103',
 'Eq104',
 'Eq105',
 'Eq106',
 'Eq107',
 'Eq108',
 'Eq109',
 'Eq110',
 'Eq111'

## 定义方程模板

In [11]:
    if Eqid == "Eq11": #建立定义模型
        y = "gdp3n_bj"
        x_list =["gdp3r_bj","gdp3d_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp3n_bj","gdp3r_bj","gdp3d_bj"]]
            #test1 = test1[(test1.index>1996) & (test1.index <2018)]
            test1[y] = (test1["gdp3r_bj"] * test1["gdp3d_bj"])/100     
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 

NameError: name 'Eqid' is not defined

## OLS模板

In [ ]:
    if Eqid == "Eq12":       
        y = "gdp2_indar_bj"
        x_list = ["consr_bj","investr_bj","exusd_bj","exrate_cn","d_gdp2_indar_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp2_indar_bj","consr_bj","investr_bj","exusd_bj","exrate_cn","d_gdp2_indar_bj"]]
            test1 = test1[(test1.index>1995) & (test1.index <2019)]
            results = sm.ols(formula = "np.log(gdp2_indar_bj) ~ np.log(consr_bj)  + np.log(investr_bj) + np.log(exusd_bj * exrate_cn) + d_gdp2_indar_bj",data = test1 ).fit()           
            test1[y] = np.exp(results.predict())              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 

## SARMAX模板

In [ ]:
    if Eqid == "Eq13": 
        y = "gdp2_bindar_bj"
        x_list = ["investr_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp2_bindar_bj","investr_bj"]]
            test1 = test1[(test1.index>1996) & (test1.index <2019)]
            ar = (0,1) 
            ma = 0
            mod = SARIMAX(endog = np.log(test1["gdp2_bindar_bj"])  , exog = np.log(test1["investr_bj"]) ,order =(ar,0,ma),enforce_invertibility= False , trend = 'c', enforce_stationarity = False)
            results = mod.fit()
            test1[y] = np.exp(results.predict())[2:]  # 和AR系数有关             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 

In [8]:
exogs = exogs_all.copy() # 列表
df_ev = data_ev.copy() #数据源 ： 数据框
preds =  pd.DataFrame(index =data_ev.index)  # 空数据框

# 北京联立方程

In [14]:
exogs_bj = [x for x in exogs_all if "_bj" in x ]
len(exogs_bj)

99

In [15]:
exogs = exogs_bj.copy()
df_ev = data_ev.copy() #数据源 ： 数据框
preds =  pd.DataFrame(index =data_ev.index)  # 空数据框
der_df =  pd.DataFrame(index =data_ev.index)  # 空数据框

## 定义方程函数

In [21]:
def callEq(Eqid): #,exogs = exogs ,df_ev = df_ev, preds = preds
    global exogs
    global df_ev
    global preds
    global Eqs
    #print(len(exogs))
    #print(df_ev.shape)   
    #print(preds.shape)
    
    if Eqid == "Eq1":
        #-----1.判断内生变量还未生成，并且自变量数据完整；
        y = "gdp1r_bj"
        x_list = ["cul_area_bj","d_gdp1r_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
            #-----2 :建立OLS模型
            test1 = df_ev[["gdp1r_bj","cul_area_bj","d_gdp1r_bj"]]
            test1["gdplr_bj_s1"] = test1["gdp1r_bj"].shift(1)  ##直接写进表达式，结果有差异
            test1 = test1[(test1.index>1996) & (test1.index <2018)]
            results = sm.ols(formula = "gdp1r_bj ~ cul_area_bj  + gdplr_bj_s1 + d_gdp1r_bj",data = test1).fit()
            # 衍生变量
            
            #------3 : 写结果
            test1[y] = results.predict()
               # 预测结果
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)
               # 更新数据源相应外生变量，方便后面作为变量进行其他模型拟合
            df_ev[y] = preds[y]
               # 将该内生变量从内生变量清单中删除 ，已证明内生变量只会计算一次 ：
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )         

        
    if Eqid == "Eq2": #建立定义模型
            y = "gdpn_bj"
            x_list = ["gdp1n_bj","gdp2n_bj","gdp3n_bj"]
            if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
                
           # try :
                #-----2 :
                test1 = df_ev[["gdpn_bj","gdp1n_bj","gdp2n_bj","gdp3n_bj"]]
                #test1 = test1[(test1.index>1996) & (test1.index <2018)]
                #------3 : 写结果
                test1[y] = test1["gdp1n_bj"] + test1["gdp2n_bj"] + test1["gdp3n_bj"]
                # 预测结果
                preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)
                # 更新数据源相应外生变量，方便后面作为变量进行其他模型拟合
                df_ev[y] = preds[y]
                # 将该内生变量从内生变量清单中删除 ，已证明内生变量只会计算一次 ：
                exogs.remove(y)
                Eqs.remove(Eqid)
                print(Eqid ,y,"----------------   求解成功 。")
            else:
                print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 

    if Eqid == "Eq3": #建立定义模型
            y = "gdpr_bj"
            x_list =["gdp1r_bj","gdp2r_bj","gdp3r_bj"]
            if (y in exogs) &( len([x for x in x_list if x in exogs])==0): 
           # try :                
                test1 = df_ev[["gdpr_bj","gdp1r_bj","gdp2r_bj","gdp3r_bj"]]
                #test1 = test1[(test1.index>1996) & (test1.index <2018)]                
                test1[y] = test1["gdp1r_bj"] + test1["gdp2r_bj"] + test1["gdp3r_bj"]             
                preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
                df_ev[y] = preds[y]                 
                exogs.remove(y)
                Eqs.remove(Eqid)
                print(Eqid ,y," ----------------   求解成功 。")
            else:
                print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
    if Eqid == "Eq4": #建立定义模型
            y = "gdp1n_bj"
            x_list =["gdp1r_bj","gdp1d_bj"]
            if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
           # try :                
                test1 = df_ev[["gdp1n_bj","gdp1r_bj","gdp1d_bj"]]
                #test1 = test1[(test1.index>1996) & (test1.index <2018)]
                test1["y"] = test1["gdp1r_bj"]*test1["gdp1d_bj"]/100            
                preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
                df_ev[y] = preds[y]                 
                exogs.remove(y)
                Eqs.remove(Eqid)
                print(Eqid ,y," ----------------   求解成功 。")
            else:
                print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )    
    if Eqid == "Eq5": #建立定义模型
        y = "gdp2_indan_bj"
        x_list =["gdp2_indar_bj","gdp2_indad_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp2_indan_bj","gdp2_indar_bj","gdp2_indad_bj"]]
            #test1 = test1[(test1.index>1996) & (test1.index <2018)]
            test1[y] = test1["gdp2_indar_bj"]*test1["gdp2_indad_bj"]/100           
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 

    if Eqid == "Eq6": #建立定义模型
        y = "gdp2_bindan_bj"
        x_list =["gdp2_bindar_bj","gdp2_bindad_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp2_bindan_bj","gdp2_bindar_bj","gdp2_bindad_bj"]]
            test1 = test1[(test1.index>1996) & (test1.index <2018)]
            test1[y] = test1["gdp2_bindar_bj"]*test1["gdp2_bindad_bj"]/100          
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )
            
    if Eqid == "Eq7": #建立定义模型
        y = "gdp2n_bj"
        x_list =["gdp2r_bj","gdp2d_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp2n_bj","gdp2r_bj","gdp2d_bj"]]
            #test1 = test1[(test1.index>1996) & (test1.index <2018)]
            test1[y] = test1["gdp2r_bj"]*test1["gdp2d_bj"]/100        
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq8": #建立定义模型
        y = "gdp2r_bj2"
        x_list =["gdp2_indar_bj","gdp2_bindar_bj","gdp2r_erro_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp2r_bj2","gdp2_indar_bj","gdp2_bindar_bj","gdp2r_erro_bj"]]
            #test1 = test1[(test1.index>1996) & (test1.index <2018)]
            test1[y] = test1["gdp2_indar_bj"] + test1["gdp2_bindar_bj"] + test1["gdp2r_erro_bj"]       
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 

    if Eqid == "Eq9":       
        y = "gdp2r_bj1"
        x_list = ["gdp_tecn_bj","gdp2d_bj","d_gdp2r_bj_1"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp2r_bj1","gdp_tecn_bj","gdp2d_bj","d_gdp2r_bj_1"]]
            test1["gdp2r_bj1_s1"] = test1["gdp2r_bj1"].shift(1)
            test1["gdp_tecn_gdp2d"] = test1["gdp_tecn_bj"]/test1["gdp2d_bj"]
            test1 = test1[(test1.index>2003) & (test1.index <2019)]
            results = sm.ols(formula = "gdp2r_bj1 ~ gdp_tecn_gdp2d  + gdp2r_bj1_s1 + d_gdp2r_bj_1",data = test1 ).fit()            
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq10": #建立定义模型
        y = "gdp2r_bj"
        x_list =["gdp2r_bj1","gdp2r_bj2"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp2r_bj","gdp2r_bj1","gdp2r_bj2"]]
            #test1 = test1[(test1.index>1996) & (test1.index <2018)]
            test1[y] = (test1["gdp2r_bj1"] + test1["gdp2r_bj2"])/2      
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )

    if Eqid == "Eq11": #建立定义模型
        y = "gdp3n_bj"
        x_list =["gdp3r_bj","gdp3d_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp3n_bj","gdp3r_bj","gdp3d_bj"]]
            #test1 = test1[(test1.index>1996) & (test1.index <2018)]
            test1[y] = (test1["gdp3r_bj"] * test1["gdp3d_bj"])/100     
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 

    if Eqid == "Eq12":       
        y = "gdp2_indar_bj"
        x_list = ["consr_bj","investr_bj","exusd_bj","exrate_cn","d_gdp2_indar_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp2_indar_bj","consr_bj","investr_bj","exusd_bj","exrate_cn","d_gdp2_indar_bj"]]
            test1 = test1[(test1.index>1995) & (test1.index <2019)]
            results = sm.ols(formula = "np.log(gdp2_indar_bj) ~ np.log(consr_bj)  + np.log(investr_bj) + np.log(exusd_bj * exrate_cn) + d_gdp2_indar_bj",data = test1 ).fit()           
            test1[y] = np.exp(results.predict())              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )   
            
    if Eqid == "Eq13": 
        y = "gdp2_bindar_bj"
        x_list = ["investr_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp2_bindar_bj","investr_bj"]]
            test1 = test1[(test1.index>1996) & (test1.index <2019)]
            ar = (0,1) 
            ma = 0
            mod = SARIMAX(endog = np.log(test1["gdp2_bindar_bj"])  , exog = np.log(test1["investr_bj"]) ,order =(ar,0,ma),enforce_invertibility= False , trend = 'c', enforce_stationarity = False)
            results = mod.fit()
            test1[y] = np.exp(results.predict())[2:]  # 和AR系数有关             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )   
            
    if Eqid == "Eq14":       
        y = "gdp3r_bj"
        x_list = ["gdp2r_bj","exusd_bj","exrate_cn"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp3r_bj","gdp2r_bj","exusd_bj","exrate_cn"]]
            test1["exusexrte"] = test1["exusd_bj"]*test1["exrate_cn"]
            test1["gdp3r_bj_s1"] = test1["gdp3r_bj"].shift(1)           
            test1 = test1[(test1.index>1996) & (test1.index <2019)]
            results = sm.ols(formula = "np.log(gdp3r_bj) ~ np.log(gdp2r_bj)  + np.log(exusexrte) +  np.log(gdp3r_bj_s1)",data = test1 ).fit()
            test1[y] = np.exp(results.predict())              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
           
    if Eqid == "Eq15": #建立定义模型
        y = "con_fr_bj"
        x_list =["con_pr_bj" , "con_gr_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["con_fr_bj" ,"con_pr_bj" , "con_gr_bj"]]
            test1[y] = (test1["con_pr_bj"] + test1["con_gr_bj"])   
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )   
    if Eqid == "Eq16": #建立定义模型
        y = "con_gr_bj"
        x_list =["con_gn_bj" , "pcon_gd_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["con_gr_bj" ,"con_gn_bj" , "pcon_gd_bj"]]
            test1[y] = (test1["con_gn_bj"] /test1["pcon_gd_bj"])*100  
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq17": #建立定义模型
        y = "con_pr_bj"
        x_list =["con_pur_bj" , "con_prr_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["con_pr_bj" ,"con_pur_bj" , "con_prr_bj"]]
            test1[y] = (test1["con_pur_bj"] + test1["con_prr_bj"]) 
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )
    if Eqid == "Eq18": #建立定义模型
        y = "con_fn_bj"
        x_list =["con_pn_bj" , "con_gn_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["con_fn_bj" ,"con_pn_bj" , "con_gn_bj"]]
            test1["预测"] = (test1["con_pn_bj"] + test1["con_gn_bj"])
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )
    if Eqid == "Eq19": #建立定义模型
        y = "con_pn_bj"
        x_list =["con_pun_bj" , "con_prn_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["con_pn_bj" ,"con_pun_bj" , "con_prn_bj"]]
            test1[y] = (test1["con_pun_bj"] + test1["con_prn_bj"])
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )
            
    if Eqid == "Eq20": #建立定义模型
        y = "con_pun_bj"
        x_list =["con_pur_bj","pcon_pud_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["con_pun_bj" ,"con_pur_bj","pcon_pud_bj"]]
            test1[y] = (test1["con_pur_bj"] * test1["pcon_pud_bj"])/100
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq21": #建立定义模型
        y = "con_prn_bj"
        x_list =["con_prr_bj" , "pcon_prd_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["con_prn_bj" ,"con_prr_bj" , "pcon_prd_bj"]]
            test1[y] = (test1["con_prr_bj"] * test1["pcon_prd_bj"])/100
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )      
            
    if Eqid == "Eq22":       
        y = "con_gn_bj"
        x_list = ["fen_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["con_gn_bj","fen_bj"]]
            test1 = test1[(test1.index>1996) & (test1.index <2019)]
            results = sm.ols(formula = "con_gn_bj ~ fen_bj",data = test1).fit()
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq23": #建立定义模型
        y = "investn_bj"
        x_list =["investr_bj" , "faipi_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["investn_bj" ,"investr_bj" , "faipi_bj"]]
            test1[y] = (test1["investr_bj"] * test1["faipi_bj"]) /100
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
            
    if Eqid == "Eq24": #建立定义模型
        y = "inv_fon_bj"
        x_list =["inv_for_bj" , "pinv_fod_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["inv_fon_bj" ,"inv_for_bj" , "pinv_fod_bj"]]
            test1[y] = (test1["inv_for_bj"] * test1["pinv_fod_bj"])/100
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )
                  
    if Eqid == "Eq25": #建立定义模型
        y = "inv_chn_bj"
        x_list =["inv_chr_bj" , "pinv_chd_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["inv_chn_bj" ,"inv_chr_bj" , "pinv_chd_bj"]]
            test1[y] = (test1["inv_chr_bj"] * test1["pinv_chd_bj"])/100
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])
                  
    if Eqid == "Eq26": #建立定义模型
        y = "consr_bj"
        x_list =["consn_bj" , "rpi_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["consr_bj" ,"consn_bj" , "rpi_bj"]]
            test1[y] = (test1["consn_bj"] / test1["rpi_bj"])*100
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])
            
    if Eqid == "Eq27": #建立定义模型
        y = "inv_for_bj"
        x_list =["inv_fir_bj" , "inv_chr_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["inv_for_bj" ,"inv_fir_bj" , "inv_chr_bj"]]
            test1[y] = test1["inv_fir_bj"] + test1["inv_chr_bj"]
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])           
 
    if Eqid == "Eq28": #建立定义模型
        y = "inv_fin_bj"
        x_list =["inv_fir_bj" , "pinv_fid_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["inv_fin_bj" ,"inv_fir_bj" , "pinv_fid_bj"]]
            test1[y] = (test1["inv_fir_bj"] * test1["pinv_fid_bj"])/100
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])  
            
    if Eqid == "Eq29": #建立定义模型
        y = "gdper_bj"
        x_list =["con_fr_bj","inv_for_bj","nex_gsr_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdper_bj" ,"con_fr_bj","inv_for_bj","nex_gsr_bj"]]
            test1[y] = test1["con_fr_bj"] + test1["inv_for_bj"] + test1["nex_gsr_bj"]
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])    
            
    if Eqid == "Eq30": #建立定义模型
        y = "gdpen_bj"
        x_list =["con_fn_bj" , "inv_fon_bj","nex_gsn_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdpen_bj" ,"con_fn_bj" , "inv_fon_bj","nex_gsn_bj"]]
            test1[y] = test1["con_fn_bj"] + test1["inv_fon_bj"] + test1["nex_gsn_bj"]
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs]) 
            
    if Eqid == "Eq31": #建立定义模型
        y = "nex_gsn_bj"
        x_list =["nex_gsr_bj" , "pnex_gsd_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["nex_gsn_bj" ,"nex_gsr_bj" , "pnex_gsd_bj"]]
            test1[y] = (test1["nex_gsr_bj"] * test1["pnex_gsd_bj"])/100
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])             
            
    if Eqid == "Eq32":       
        y = "con_pur_bj"
        x_list = ["yht_un_bj" , "pcon_pud_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["con_pur_bj" ,"yht_un_bj" , "pcon_pud_bj"]]
            test1["yht_pcon_100"] = test1["yht_un_bj"]/test1["pcon_pud_bj"]*100
            test1 = test1[(test1.index>2005) & (test1.index <2019)]
            results = sm.ols(formula = "con_pur_bj ~ yht_pcon_100",data = test1 ).fit()           
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq33":       
        y = "con_prr_bj"
        x_list = ["yht_rn_bj" , "pcon_prd_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["con_prr_bj" ,"yht_rn_bj" , "pcon_prd_bj"]]
            test1["yht_pcon_100"] = test1["yht_rn_bj"]/test1["pcon_prd_bj"]*100
            test1["diff_con_prr"] = test1["con_prr_bj"].shift(1)-test1["con_prr_bj"].shift(2)
            test1 = test1[(test1.index>2007) & (test1.index <2019)]
            results = sm.ols(formula = "con_prr_bj ~ yht_pcon_100 + diff_con_prr",data = test1 ).fit()          
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 


    if Eqid == "Eq34":       
        y = "investr_bj"
        x_list = ["loann_bj" , "faipi_bj","d_investr_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["investr_bj" ,"loann_bj" , "faipi_bj","d_investr_bj"]]
            test1["loan_fai_100"] = test1["loann_bj"]/test1["faipi_bj"]*100
            test1 = test1[(test1.index>2005) & (test1.index <2019)]
            results = sm.ols(formula = "investr_bj ~ loan_fai_100 + d_investr_bj",data = test1 ).fit()          
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq35":       
        y = "inv_fir_bj"
        x_list = ["investr_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["inv_fir_bj" ,"investr_bj" ]]
            test1["inv_fir_bj_s1"] = test1["inv_fir_bj"].shift(1)
            test1 = test1[(test1.index>2006) & (test1.index <2019)]
            results = sm.ols(formula = "inv_fir_bj ~ investr_bj + inv_fir_bj_s1",data = test1 ).fit()           
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq36":       
        y = "nex_gsr_bj"
        x_list = ["gdpr_bj" ,"d_nex_gsr_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["nex_gsr_bj","gdpr_bj" ,"d_nex_gsr_bj" ]]
            test1["nex_gsr_bj_s1"] = test1["nex_gsr_bj"].shift(1)
            test1 = test1[(test1.index>2006) & (test1.index <2019)]
            results = sm.ols(formula = "nex_gsr_bj ~ gdpr_bj + nex_gsr_bj_s1 + d_nex_gsr_bj",data = test1 ).fit()          
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
    if Eqid == "Eq37":       
        y = "consn_bj"
        x_list = ["yht_n_bj" , "d_consn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["consn_bj" ,"yht_n_bj" , "d_consn_bj"]]
            test1["yht_n_bj_s1"] = test1["yht_n_bj"].shift(1)
            test1["consn_bj_s1"] = test1["consn_bj"].shift(1)
            test1["n_s1_diff"] = test1["yht_n_bj"] - test1["yht_n_bj_s1"]
            results = sm.ols(formula = "consn_bj ~ n_s1_diff +consn_bj_s1 + d_consn_bj",data = test1 ).fit()           
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq38":       
        y = "exusd_bj"
        x_list = ["exusd_ch" , "d_exusd_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["exusd_bj" ,"exusd_ch" , "d_exusd_bj"]]
            test1["exusd_bj_s1"] = test1["exusd_bj"].shift(1) 
            test1["exusd_bj_diff"] = test1["exusd_bj"].shift(1) - test1["exusd_bj"].shift(2)
            test1 = test1[(test1.index>1997) & (test1.index <2019)]  # 本来没这行 。但不加的话，可能预测值对不齐
            results = sm.ols(formula = "exusd_bj ~ exusd_ch +exusd_bj_s1 + exusd_bj_diff + d_exusd_bj",data = test1 ).fit()         
            test1[y] = results.predict() 
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 

    if Eqid == "Eq39":       
        y = "imusd_bj"
        x_list = ["exusd_bj" , "d_imusd_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["imusd_bj" ,"exusd_bj" , "d_imusd_bj"]]
            test1["imusd_bj_s1"] = test1["imusd_bj"].shift(1)
            test1 = test1[(test1.index>1997) & (test1.index <2019)]  # 本来没这行 。但不加的话，可能预测值对不齐
            results = sm.ols(formula = "np.log(imusd_bj) ~ np.log(imusd_bj_s1) + np.log(exusd_bj) + d_imusd_bj",data = test1 ).fit()                      
            test1[y] = np.exp(results.predict())              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq40": #建立定义模型
        y = "yht_n_bj"
        x_list =["yht_rn_bj","yht_un_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["yht_n_bj" ,"yht_rn_bj","yht_un_bj"]]
            test1[y] = test1["yht_rn_bj"] + test1["yht_un_bj"]
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])            
    if Eqid == "Eq41": #建立定义模型
        y = "yh_un_bj"
        x_list =["yht_un_bj","popu_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["yh_un_bj" ,"yht_un_bj","popu_bj"]]
            test1[y] = (test1["yht_un_bj"] / test1["popu_bj"])
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])
            
    if Eqid == "Eq42": #建立定义模型
        y = "yh_rn_bj"
        x_list =["yht_rn_bj","popr_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["yh_rn_bj" ,"yht_rn_bj","popr_bj"]]
            test1[y] = (test1["yht_rn_bj"] / test1["popr_bj"])
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])

    if Eqid == "Eq43": #建立定义模型
        y = "yhn_bj"
        x_list =["yht_n_bj","totpop_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["yhn_bj" ,"yht_n_bj","totpop_bj"]]
            test1[y] = (test1["yht_n_bj"] / test1["totpop_bj"])
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])            
            

    if Eqid == "Eq44": #建立定义模型
        y = "conet_un_bj"
        x_list =["cone_un_bj","popu_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["conet_un_bj" ,"cone_un_bj","popu_bj"]]
            test1[y] = test1["cone_un_bj"] * test1["popu_bj"]
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])            
    if Eqid == "Eq45": #建立定义模型
        y = "conet_rn_bj"
        x_list =["cone_rn_bj","popr_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["conet_rn_bj" ,"cone_rn_bj","popr_bj"]]
            test1[y] = test1["cone_rn_bj"] * test1["popr_bj"]
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])
            
            

    if Eqid == "Eq46":       
        y = "yht_un_bj"
        x_list = ["gdp2n_bj","gdp3n_bj","taxn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["yht_un_bj","gdp2n_bj","gdp3n_bj","taxn_bj"]]
            test1["yht_un_bj_s1"] = test1["yht_un_bj"].shift(1)
            test1["gdp2gdp3tax"] = test1["gdp2n_bj"] + test1["gdp3n_bj"] - test1["taxn_bj"]
            results = sm.ols(formula = "yht_un_bj ~ gdp2gdp3tax + yht_un_bj_s1",data = test1 ).fit()          
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 

    if Eqid == "Eq47":       
        y = "yht_rn_bj"
        x_list = ["gdp1n_bj","gdp3n_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["yht_rn_bj","gdp1n_bj","gdp3n_bj"]]
            test1["yht_rn_bj_s1"] = test1["yht_rn_bj"].shift(1)
            results = sm.ols(formula = "yht_rn_bj ~ gdp1n_bj + gdp3n_bj + yht_rn_bj_s1",data = test1 ).fit()          
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq48":       
        y = "cone_un_bj"
        x_list = ["yht_un_bj" , "yh_un_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["cone_un_bj" , "yh_un_bj" ]]
            test1["un_diff"] = test1["yh_un_bj"] - test1["yh_un_bj"].shift(1)
            test1["cone_un_bj_s1"] = test1["cone_un_bj"].shift(1)
            results = sm.ols(formula = "cone_un_bj ~ un_diff + cone_un_bj_s1 ",data = test1 ).fit()           
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )             
            
 
    if Eqid == "Eq49":       
        y = "cone_rn_bj"
        x_list = ["yh_rn_bj" ,"d_cone_rn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["cone_rn_bj" , "yh_rn_bj" ,"d_cone_rn_bj" ]]
            results = sm.ols(formula = "cone_rn_bj ~ yh_rn_bj+ d_cone_rn_bj ",data = test1 ).fit()          
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq50":       
        y = "frn_bj"
        x_list = ["taxn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["frn_bj","taxn_bj"]]
            test1["frn_bj_s1"] = test1["frn_bj"].shift(1)
            test1 = test1[(test1.index>1997) & (test1.index <2019)]
            results = sm.ols(formula = "frn_bj ~ taxn_bj + frn_bj_s1 ",data = test1 ).fit()           
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )   
    if Eqid == "Eq51": 
        y = "taxn_bj1"
        x_list = ["tax_vadn_bj","tax_cinn_bj","tax_pinn_bj","d_taxn_bj_1"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["taxn_bj1","tax_vadn_bj","tax_cinn_bj","tax_pinn_bj","d_taxn_bj_1"]]
            test1["log_tax_sum"] =np.log( test1["tax_vadn_bj"] + test1["tax_cinn_bj"] + test1["tax_pinn_bj"])
            test1 = test1[(test1.index>1996) & (test1.index <2019)]
            ar = 1
            ma = 0
            mod = SARIMAX(endog = np.log(test1["taxn_bj1"])  , exog = test1[["log_tax_sum","d_taxn_bj_1"] ],order =(ar,0,ma),enforce_invertibility= False , trend = 'c', enforce_stationarity = False)
            results = mod.fit()
            test1[y] = np.exp(results.predict())[1:]  # 和AR系数有关             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
            
    if Eqid == "Eq52":       
        y = "taxn_bj2"
        x_list = ["gdpn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["taxn_bj2","gdpn_bj"]]
            test1["taxn_bj2_s1"] = test1["taxn_bj2"].shift(1)
            results = sm.ols(formula = "taxn_bj2 ~ gdpn_bj+ taxn_bj2_s1 ",data = test1 ).fit()         
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq53": #建立定义模型
        y = "taxn_bj"
        x_list =["taxn_bj1","taxn_bj2"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["taxn_bj","taxn_bj1","taxn_bj2"]]
            test1[y] = (test1["taxn_bj1"] + test1["taxn_bj2"])/2
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])  
            
    if Eqid == "Eq54":       
        y = "tax_vadn_bj"
        x_list = ["gdpn_bj","d_tax_vadn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["tax_vadn_bj","gdpn_bj","d_tax_vadn_bj"]]
            test1["tax_vadn_bj_s1"] = test1["tax_vadn_bj"].shift(1)
            test1["log_tax_vadn"] = np.log(test1["tax_vadn_bj"])
            results = sm.ols(formula = "np.log(tax_vadn_bj) ~ np.log(gdpn_bj) + np.log(tax_vadn_bj_s1) + d_tax_vadn_bj ",data = test1 ).fit()     
            test1[y] = np.exp(results.predict())              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )
            
    if Eqid == "Eq55":       
        y = "tax_cinn_bj"
        x_list = ["gdpn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["tax_cinn_bj","gdpn_bj"]]
            test1["tax_cinn_bj_s1"] = test1["tax_cinn_bj"].shift(1)
            test1 = test1[(test1.index>1996) & (test1.index <2019)]
            results = sm.ols(formula = "tax_cinn_bj ~ gdpn_bj+ tax_cinn_bj_s1 ",data = test1 ).fit()        
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )
            
    if Eqid == "Eq56":       
        y = "tax_pinn_bj"
        x_list = ["yht_n_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["tax_pinn_bj","yht_n_bj"]]
            test1["tax_pinn_bj_s1"] = test1["tax_pinn_bj"].shift(1)
            test1 = test1[(test1.index>1996) & (test1.index <2019)]
            results = sm.ols(formula = "tax_pinn_bj ~ yht_n_bj+ tax_pinn_bj_s1 ",data = test1 ).fit()        
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )
    if Eqid == "Eq57": 
        y = "fen_bj1"
        x_list = ["frn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["fen_bj1","frn_bj"]]
            test1 = test1[(test1.index>1997) & (test1.index <2019)]
            ar = 1 
            ma = 0
            mod = SARIMAX(endog = test1["fen_bj1"] , exog = test1["frn_bj"] ,order =(ar,0,ma),enforce_invertibility= False , trend = 'c', enforce_stationarity = False)
            results = mod.fit()
            test1[y] = results.predict()[1:]  # = AR系数有关             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )
            
            
    if Eqid == "Eq58":       
        y = "fen_bj2"
        x_list = ["fe_edun_bj","fe_tecn_bj","fe_culn_bj","fe_ssen_bj","fe_medn_bj","fe_eepn_bj","d_fen_bj_2"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["fen_bj2","fe_edun_bj","fe_tecn_bj","fe_culn_bj","fe_ssen_bj","fe_medn_bj","fe_eepn_bj","d_fen_bj_2"]]
            test1["fen_sum"] = test1[["fe_edun_bj","fe_tecn_bj","fe_culn_bj","fe_ssen_bj","fe_medn_bj","fe_eepn_bj"]].sum(axis =1)
            test1 = test1[(test1.index>2005) & (test1.index <2019)]
            results = sm.ols(formula = "fen_bj2 ~ fen_sum + d_fen_bj_2 ",data = test1 ).fit()         
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )            
    if Eqid == "Eq59": #建立定义模型
        y = "fen_bj"
        x_list =["fen_bj1","fen_bj2"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["fen_bj","fen_bj1","fen_bj2"]]
            test1[y] = (test1["fen_bj1"]+test1["fen_bj2"])/2
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])             
 
            
    if Eqid == "Eq60":       
        y = "depn_bj"
        x_list = ["dep_san_bj","d_depn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["depn_bj","dep_san_bj","d_depn_bj"]]
            test1["depn_bj_s1"] = test1["depn_bj"].shift(1)
            test1 = test1[(test1.index>2004) & (test1.index <2019)]
            results = sm.ols(formula = "np.log(depn_bj) ~ np.log(dep_san_bj) + np.log(depn_bj_s1) + d_depn_bj ",data = test1 ).fit()       
            test1[y] = np.exp(results.predict())              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
    if Eqid == "Eq61":       
        y = "dep_san_bj"
        x_list = ["yht_n_bj","d_dep_san_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["dep_san_bj","yht_n_bj","d_dep_san_bj"]]
            test1["dep_san_bj_s1"] = test1["dep_san_bj"].shift(1)
            test1 = test1[(test1.index>2001) & (test1.index <2019)]
            results = sm.ols(formula = "np.log(dep_san_bj) ~ np.log(yht_n_bj) + np.log(dep_san_bj_s1) + d_dep_san_bj ",data = test1 ).fit()

            test1[y] = np.exp(results.predict())              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )            
    if Eqid == "Eq62":       
        y = "loan_ln_bj"
        x_list = ["depn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           

            test1 = df_ev[["loan_ln_bj","depn_bj"]]
            test1["loan_ln_bj_s1"] = test1["loan_ln_bj"].shift(1)
            test1 = test1[(test1.index>2003) & (test1.index <2019)]
            results = sm.ols(formula = "np.log(loan_ln_bj) ~ np.log(depn_bj) + np.log(loan_ln_bj_s1) ",data = test1 ).fit()
            test1[y] = np.exp(results.predict())              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )             

    if Eqid == "Eq63":       
        y = "loann_bj"
        x_list = ["depn_bj","intratel_cn","d_loann_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["loann_bj","depn_bj","intratel_cn","d_loann_bj"]]
            test1["loann_bj_s1"] = test1["loann_bj"].shift(1)
            test1["depn_bj_sum"] = test1["depn_bj"] + test1["depn_bj"].shift(1)
            test1 = test1[(test1.index>2006) & (test1.index <2019)]
            results = sm.ols(formula = "loann_bj ~ depn_bj_sum + loann_bj_s1  + intratel_cn + d_loann_bj ",data = test1 ).fit()
            test1[y] = np.exp(results.predict())              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
            
    if Eqid == "Eq64": #建立定义模型
        y = "emp1_bj"
        x_list =["totemp_bj" , "emp2_bj" , "emp3_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["emp1_bj" , "totemp_bj" , "emp2_bj" , "emp3_bj"]]
            test1[y] = test1["totemp_bj"] - test1["emp2_bj"] - test1["emp3_bj"]
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])      
    if Eqid == "Eq65": #建立定义模型
        y = "popr_bj"
        x_list =["totpop_bj" , "popu_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["popr_bj" , "totpop_bj" , "popu_bj" ]]
            test1[y] = test1["totpop_bj"] - test1["popu_bj"] 
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs])            
            
    if Eqid == "Eq66":       
        y = "totemp_bj"
        x_list = ["gdpr_bj","d_totemp_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["totemp_bj","gdpr_bj","d_totemp_bj"]]
            test1["gdpr_bj_diff"] = test1["gdpr_bj"] - test1["gdpr_bj"].shift(1)
            test1["totemp_bj_s1"] = test1["totemp_bj"].shift(1)
            test1 = test1[(test1.index>1996) & (test1.index <2019)]
            results = sm.ols(formula = "totemp_bj ~ gdpr_bj_diff + totemp_bj_s1  + d_totemp_bj ",data = test1 ).fit()
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
    if Eqid == "Eq67":       
        y = "emp2_bj"
        x_list = ["gdpr_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["emp2_bj","gdpr_bj"]]
            test1["gdpr_bj_pch"] =  (test1["gdpr_bj"] - test1["gdpr_bj"].shift(1) )/  test1["gdpr_bj"].shift(1)  #增长率
            test1["emp2_bj_pch"] = (test1["emp2_bj"] - test1["emp2_bj"].shift(1) )/  test1["emp2_bj"].shift(1)
            test1["emp2_bj_s1"] = test1["emp2_bj"].shift(1)
            test1 = test1[(test1.index>1998) & (test1.index <2019)]
            results = sm.ols(formula = "emp2_bj ~ gdpr_bj + gdpr_bj_pch  + emp2_bj_pch + emp2_bj_s1 ",data = test1 ).fit()
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
            
    if Eqid == "Eq68":       
        y = "emp3_bj"
        x_list = ["gdp3r_bj" ,"d_emp3_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["emp3_bj","gdp3r_bj" ,"d_emp3_bj"]]
            test1["gdp3r_bj_pch"] =  (test1["gdp3r_bj"] - test1["gdp3r_bj"].shift(1) )/  test1["gdp3r_bj"].shift(1)  #增长率
            test1["emp3_bj_s1"] = test1["emp3_bj"].shift(1)
            test1 = test1[(test1.index>1997) & (test1.index <2019)]
            results = sm.ols(formula = "emp3_bj ~ gdp3r_bj_pch + emp3_bj_s1  + d_emp3_bj",data = test1 ).fit()
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq69":       
        y = "totpop_bj"
        x_list = ["gdpr_bj","d_totpop_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["totpop_bj","gdpr_bj","d_totpop_bj"]]
            test1["gdpr_bj_diff"] = test1["gdpr_bj"] - test1["gdpr_bj"].shift(1)
            test1["totpop_bj_s1"] = test1["totpop_bj"].shift(1)
            test1 = test1[(test1.index>1997) & (test1.index <2019)]
            results = sm.ols(formula = "totpop_bj ~ gdpr_bj_diff + totpop_bj_s1  + d_totpop_bj",data = test1 ).fit()
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )              
    if Eqid == "Eq70": 
        y = "popu_bj"
        x_list = ["d_popu_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["popu_bj" , "d_popu_bj" ]]
            test1["popu_bj_s1"] = test1["popu_bj"].shift(1)
            test1 = test1[(test1.index>1997) & (test1.index <2019)]
            ar = 1
            ma = 0
            mod = SARIMAX(endog = test1["popu_bj"]  , exog = test1[["popu_bj_s1","d_popu_bj"]] ,order =(ar,0,ma),enforce_invertibility= False , trend = 'c', enforce_stationarity = False)
            results = mod.fit()
            test1[y] = results.predict()[1:]  # 和AR系数有关             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 

    if Eqid == "Eq71":       
        y = "rpi_bj"
        x_list = ["cpi_bj","d_rpi_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["rpi_bj","cpi_bj","d_rpi_bj"]]
            test1["cpi_bj_diff"] = test1["cpi_bj"] - test1["cpi_bj"].shift(1)
            test1["rpi_bj_s1"] = test1["rpi_bj"].shift(1)
            test1 = test1[(test1.index>1997) & (test1.index <2019)]
            results = sm.ols(formula = "rpi_bj ~ cpi_bj_diff + rpi_bj_s1  + d_rpi_bj",data = test1 ).fit()
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )              
    if Eqid == "Eq72": 
        y = "cpi_bj"
        x_list = ["gdpd_bj" ,"d_cpi_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["cpi_bj","gdpd_bj" ,"d_cpi_bj"]]
            test1["cpi_bj_s1"] = test1["cpi_bj"].shift(1)
            test1["gdpd_bj_diff"] = test1["gdpd_bj"] - test1["gdpd_bj"].shift(1)
            test1 = test1[(test1.index>1997) & (test1.index <2019)]
            ar = 1
            ma = 0
            mod = SARIMAX(endog = test1["cpi_bj"]  , exog = test1[["cpi_bj_s1","gdpd_bj_diff","d_cpi_bj"]] ,order =(ar,0,ma),enforce_invertibility= False , trend = 'c', enforce_stationarity = False)
            results = mod.fit()
            test1[y] = np.exp(results.predict())[1:]  # 和AR系数有关             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
    if Eqid == "Eq73":       
        y = "gdp1d_bj"
        x_list = ["rpi_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp1d_bj","rpi_bj"]]
            test1["rpi_bj_diff"] = test1["rpi_bj"] - test1["rpi_bj"].shift(1)
            test1["gdp1d_bj_s1"] = test1["gdp1d_bj"].shift(1)
            test1 = test1[(test1.index>1997) & (test1.index <2019)]
            results = sm.ols(formula = "gdp1d_bj ~ rpi_bj_diff + gdp1d_bj_s1",data = test1 ).fit()
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )   
    if Eqid == "Eq74":       
        y = "gdp2d_bj"
        x_list = ["ppi_bj","faipi_bj","d_gdp2d_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp2d_bj","ppi_bj","faipi_bj","d_gdp2d_bj"]]
            test1 = test1[(test1.index>1995) & (test1.index <2019)]
            results = sm.ols(formula = "np.log(gdp2d_bj) ~ np.log(ppi_bj) + np.log(faipi_bj) + d_gdp2d_bj",data = test1 ).fit()
            test1[y] = np.exp(results.predict())           
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )              
    if Eqid == "Eq75":       
        y = "gdp3d_bj"
        x_list = ["gdp2d_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp3d_bj","gdp2d_bj"]]
            test1["gdp3d_bj_s1"] = test1["gdp3d_bj"].shift(1)
            test1 = test1[(test1.index>1996) & (test1.index <2019)]
            results = sm.ols(formula = "gdp3d_bj ~ gdp2d_bj + gdp3d_bj_s1",data = test1 ).fit()
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )              
    if Eqid == "Eq76":       
        y = "gdpd_bj"
        x_list = ["gdp1d_bj","gdp2d_bj","gdp3d_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdpd_bj","gdp1d_bj","gdp2d_bj","gdp3d_bj"]]
            test1 = test1[(test1.index>1995) & (test1.index <2019)]
            results = sm.ols(formula = "gdpd_bj ~ gdp1d_bj + gdp2d_bj + gdp3d_bj",data = test1 ).fit()
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
    if Eqid == "Eq77":       
        y = "ppi_bj"
        x_list = ["rpi_bj","d_ppi_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["ppi_bj","rpi_bj","d_ppi_bj"]]
            test1["rpi_bj_diff"] = np.log(test1["rpi_bj"])- np.log(test1["rpi_bj"].shift(1))
            test1["ppi_bj_s1"] = test1["ppi_bj"].shift(1)
            test1 = test1[(test1.index>1996) & (test1.index <2019)]
            results = sm.ols(formula = "np.log(ppi_bj) ~ rpi_bj_diff + np.log(ppi_bj_s1)  + d_ppi_bj ",data = test1 ).fit()
            test1[y] = np.exp(results.predict() )             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
    if Eqid == "Eq78":       
        y = "faipi_bj"
        x_list = ["ppi_bj","d_faipi_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["faipi_bj","ppi_bj","d_faipi_bj"]]
            test1["ppi_bj_diff"] = test1["ppi_bj"] - test1["ppi_bj"].shift(1)
            test1["faipi_bj_s1"] = test1["faipi_bj"].shift(1)
            test1 = test1[(test1.index>1996) & (test1.index <2019)]
            results = sm.ols(formula = "np.log(faipi_bj) ~ ppi_bj_diff + np.log(faipi_bj_s1)  + d_faipi_bj ",data = test1 ).fit()
            test1[y] = np.exp(results.predict())              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
            
    if Eqid == "Eq79": #建立定义模型
        y = "gdp3_tspar_bj"
        x_list =["gdp3_tspan_bj","pgdp3_tspad_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp3_tspar_bj","gdp3_tspan_bj","pgdp3_tspad_bj"]]
            test1[y] = test1["gdp3_tspan_bj"] / test1["pgdp3_tspad_bj"] *100    
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
    if Eqid == "Eq80":       
        y = "gdp3_tspan_bj"
        x_list = ["gdp3_tratp_bj","gdp3_tragt_bj","gdp3_trapt_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp3_tspan_bj","gdp3_tratp_bj","gdp3_tragt_bj","gdp3_trapt_bj"]]
            test1 = test1[(test1.index>1999) & (test1.index <2019)]
            results = sm.ols(formula = " gdp3_tspan_bj~ gdp3_tratp_bj + gdp3_tragt_bj  + gdp3_trapt_bj ",data = test1 ).fit()
            test1[y] = results.predict()             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )             
            
            
    if Eqid == "Eq81": #建立定义模型
        y = "gdp3_wrar_bj"
        x_list =["gdp3_wran_bj","pgdp3_wrad_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp3_wrar_bj","gdp3_wran_bj","pgdp3_wrad_bj"]]
            test1[y] = test1["gdp3_wran_bj"] / test1["pgdp3_wrad_bj"] *100    
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )             
    if Eqid == "Eq82":       
        y = "gdp3_wran_bj"
        x_list = ["consn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp3_wran_bj","consn_bj" ]]
            test1["consn_bj_diff"] = test1["consn_bj"] - test1["consn_bj"].shift(1)
            test1["gdp3_wran_bj_s1"] = test1["gdp3_wran_bj"].shift(1)
            test1 = test1[(test1.index>2000) & (test1.index <2019)]
            results = sm.ols(formula = " gdp3_wran_bj~ consn_bj_diff + gdp3_wran_bj_s1 ",data = test1 ).fit()
            test1[y] = results.predict()             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
    if Eqid == "Eq83": #建立定义模型
        y = "gdp3_fiar_bj"
        x_list =["gdp3_fian_bj","pgdp3_fiad_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp3_fiar_bj","gdp3_fian_bj","pgdp3_fiad_bj"]]
            test1[y] = test1["gdp3_fian_bj"] / test1["pgdp3_fiad_bj"] *100    
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
            
    if Eqid == "Eq84":       
        y = "gdp3_fian_bj"
        x_list = ["depn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp3_fian_bj","depn_bj"]]
            test1["gdp3_fian_bj_s1"] = test1["gdp3_fian_bj"].shift(1)
            test1 = test1[(test1.index>2003) & (test1.index <2019)]
            results = sm.ols(formula = " gdp3_fian_bj~ depn_bj + gdp3_fian_bj_s1 ",data = test1 ).fit()
            test1[y] = results.predict()             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq85": 
        y = "gdp3_rear_bj"
        x_list =["gdp3_rean_bj","pgdp3_read_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp3_rear_bj","gdp3_rean_bj","pgdp3_read_bj"]]
            test1[y] = test1["gdp3_rean_bj"] / test1["pgdp3_read_bj"] *100    
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq86":       
        y = "gdp3_rean_bj"
        x_list = ["inv_rean_bj","d_gdp3_rean_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp3_rean_bj","inv_rean_bj","d_gdp3_rean_bj"]]
            test1["gdp3_rean_bj_s1"] = test1["gdp3_rean_bj"].shift(1)
            test1 = test1[(test1.index>2002) & (test1.index <2019)]
            results = sm.ols(formula = " gdp3_rean_bj~ inv_rean_bj + gdp3_rean_bj_s1 + d_gdp3_rean_bj ",data = test1 ).fit()            
            test1[y][1:] = results.predict()             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )             
    if Eqid == "Eq87": 
        y = "gdp3_infr_bj"
        x_list =["gdp3_infn_bj","pgdp3_infd_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp3_infr_bj","gdp3_infn_bj","pgdp3_infd_bj"]]
            test1[y] = test1["gdp3_infn_bj"] / test1["pgdp3_infd_bj"] *100  
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
    if Eqid == "Eq88":       
        y = "gdp3_infn_bj"
        x_list = ["gdp_tran_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp3_infn_bj","gdp_tran_bj"]]
            test1 = test1[(test1.index>2014) & (test1.index <2019)]
            results = sm.ols(formula = " gdp3_infn_bj~ gdp_tran_bj ",data = test1 ).fit()
            test1[y] = results.predict()             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
    if Eqid == "Eq89": 
        y = "gdp3_renr_bj"
        x_list =["gdp3_renn_bj","pgdp3_rend_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp3_renr_bj","gdp3_renn_bj","pgdp3_rend_bj"]]
            test1[y] = test1["gdp3_renn_bj"] / test1["pgdp3_rend_bj"] *100
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")  
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )    
            
    if Eqid == "Eq90":       
        y = "gdp3_renn_bj"
        x_list = ["gdp_momn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp3_renn_bj","gdp_momn_bj"]]
            test1 = test1[(test1.index>2005) & (test1.index <2019)]
            results = sm.ols(formula = " gdp3_renn_bj~ gdp_momn_bj ",data = test1 ).fit()
            print("Eq90: " ,test1.shape , len(results.predict()))
            test1[y] = results.predict()             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
    if Eqid == "Eq91": 
        y = "gdp3_sicr_bj"
        x_list =["gdp3_sicn_bj","pgdp3_sicd_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp3_sicr_bj","gdp3_sicn_bj","pgdp3_sicd_bj"]]
            test1[y] = test1["gdp3_sicn_bj"] / test1["pgdp3_sicd_bj"] *100 
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")            
    if Eqid == "Eq92":       
        y = "gdp3_sicn_bj"
        x_list = ["gdp3_sicn_bj","gdp_momn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp3_sicn_bj","gdp_momn_bj"]]
            test1["gdp3_sicn_bj_s1"] = test1["gdp3_sicn_bj"].shift(1)
            test1 = test1[(test1.index>2005) & (test1.index <2019)]
            results = sm.ols(formula = " gdp3_sicn_bj~ gdp_momn_bj + gdp3_sicn_bj_s1",data = test1 ).fit()
            test1[y] = results.predict()             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )   
            
    if Eqid == "Eq93":       
        y = "gdp_tran_bj"
        x_list = ["gdp_tecn_bj","d_gdp_tran_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp_tran_bj","gdp_tecn_bj","d_gdp_tran_bj"]]
            test1 = test1[(test1.index>2014) & (test1.index <2019)]
            results = sm.ols(formula = " gdp_tran_bj~ gdp_tecn_bj + d_gdp_tran_bj",data = test1 ).fit()
            test1[y] = results.predict()             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )   
    if Eqid == "Eq94":       
        y = "gdp_tecn_bj"
        x_list = ["lse_bj","tta_bj","d_gdp_tecn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp_tecn_bj","lse_bj","tta_bj","d_gdp_tecn_bj"]]
            test1["tta_bj_diff"] = test1["tta_bj"] - test1["tta_bj"].shift(1)
            test1["gdp_tecn_bj_s1"] = test1["gdp_tecn_bj"].shift(1)
            test1 = test1[(test1.index>2004) & (test1.index <2019)]
            results = sm.ols(formula = " gdp_tecn_bj ~ lse_bj + tta_bj_diff + gdp_tecn_bj_s1 + d_gdp_tecn_bj",data = test1 ).fit()
            test1[y] = results.predict()             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
            
    if Eqid == "Eq95":       
        y = "tta_f_bj"
        x_list = ["tta_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["tta_f_bj","tta_bj"]]
            test1 = test1[(test1.index>2002) & (test1.index <2019)]
            results = sm.ols(formula = " tta_f_bj~ tta_bj ",data = test1 ).fit()
            print(test1.shape ,results.predict() )
            test1[y] = results.predict()             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )             
    if Eqid == "Eq96":       
        y = "gdp_mosn_bj"
        x_list = ["gdp3_infn_bj","gdp3_renn_bj","gdp3_sicn_bj","d_gdp_mosn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp_mosn_bj","gdp3_infn_bj","gdp3_renn_bj","gdp3_sicn_bj","d_gdp_mosn_bj"]]
            test1 = test1[(test1.index>2003) & (test1.index <2019)]
            results = sm.ols(formula = " gdp_mosn_bj ~ gdp3_infn_bj + gdp3_renn_bj + gdp3_sicn_bj + d_gdp_mosn_bj",data = test1 ).fit()
            test1[y] = results.predict()             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )             
    if Eqid == "Eq97":       
        y = "gdp_momn_bj"
        x_list = ["gdp_tecn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp_momn_bj","gdp_tecn_bj"]]
            test1["gdp_tecn_bj_diff"] = test1["gdp_tecn_bj"] - test1["gdp_tecn_bj"].shift(1)
            test1["gdp_momn_bj_s1"] = test1["gdp_momn_bj"].shift(1)
            test1 = test1[(test1.index>2005) & (test1.index <2019)]
            results = sm.ols(formula = " gdp_momn_bj ~ gdp_tecn_bj_diff + gdp_momn_bj_s1",data = test1 ).fit()
            test1[y] = results.predict()             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
    if Eqid == "Eq98":       
        y = "gdp_infn_bj"
        x_list = ["gdp_tecn_bj","d_gdp_infn_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp_infn_bj","gdp_tecn_bj","d_gdp_infn_bj"]]
            test1["gdp_infn_bj_s1"] = test1["gdp_infn_bj"].shift(1)
            test1 = test1[(test1.index>2004) & (test1.index <2019)]
            results = sm.ols(formula = " gdp_infn_bj ~ gdp_tecn_bj + gdp_infn_bj_s1 + d_gdp_infn_bj",data = test1 ).fit()
            test1[y] = results.predict()             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq99":       
        y = "tta_bj"
        x_list = ["lse_bj","d_tta_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["tta_bj","lse_bj","d_tta_bj"]]
            test1["tta_bj_diff"] = test1["tta_bj"] - test1["tta_bj"].shift(1)
            test1 = test1[(test1.index>2002) & (test1.index <2019)]
            results = sm.ols(formula = " tta_bj ~ lse_bj + tta_bj_diff + d_tta_bj",data = test1 ).fit()
            test1[y] = results.predict()             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )             
           
    return  exogs , df_ev ,preds,Eqs

## 测试方程函数
- 可能因if条件未满足，有些回归、arima方程会有没有时间段限制，报错“长短不一致”（之前未单独测，预测值）；
- 加上时间限制即可

In [22]:
exogs = exogs_all.copy()

In [24]:
Eqs = Eqlist.copy()
exogs = exogs_all.copy() # 列表
df_ev = data_ev.copy() #数据源 ： 数据框
preds =  pd.DataFrame(index =data_ev.index)  # 空数据框
for eqid in Eqs[0:99] :
    exogs,df_ev,preds,Eqs = callEq(eqid)    
del exogs,df_ev,preds

Eq1 gdp1r_bj  ----------------   求解成功 。
Eq2 gdpn_bj 还不能求解 ，因还未获得外生变量:   ['gdp1n_bj', 'gdp2n_bj', 'gdp3n_bj']
Eq3 gdpr_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj', 'gdp3r_bj']
Eq4 gdp1n_bj 还不能求解 ，因还未获得外生变量:   ['gdp1d_bj']
Eq5 gdp2_indan_bj 还不能求解 ，因还未获得外生变量:   ['gdp2_indar_bj']
Eq6 gdp2_bindan_bj 还不能求解 ，因还未获得外生变量:   ['gdp2_bindar_bj']
Eq7 gdp2n_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj', 'gdp2d_bj']
Eq8 gdp2r_bj2 还不能求解 ，因还未获得外生变量:   ['gdp2_indar_bj', 'gdp2_bindar_bj']
Eq9 gdp2r_bj1 还不能求解 ，因还未获得外生变量:   ['gdp_tecn_bj', 'gdp2d_bj']
Eq10 gdp2r_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj1', 'gdp2r_bj2']
Eq11 gdp3n_bj 还不能求解 ，因还未获得外生变量:   ['gdp3r_bj', 'gdp3d_bj']
Eq12 gdp2_indar_bj 还不能求解 ，因还未获得外生变量:   ['consr_bj', 'investr_bj', 'exusd_bj']
Eq13 gdp2_bindar_bj 还不能求解 ，因还未获得外生变量:   ['investr_bj']
Eq14 gdp3r_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj', 'exusd_bj']
Eq15 gdp3r_bj 还不能求解 ，因还未获得外生变量:   ['con_pr_bj', 'con_gr_bj']
Eq16 con_gr_bj 还不能求解 ，因还未获得外生变量:   ['con_gn_bj']
Eq17 con_pr_bj 还不能求解 ，因还未获得外生变量:   ['con_pur_bj', 'con_prr_bj'

## 方程求解一： 方程简化

- 方程简化总出现时间段对不齐情况，稍微再修正 ；
- 先测试 3SLS 是否对每个方程时间段不一样的问题；

In [25]:
Eqs = Eqlist[0:99].copy() #====>北京
exogs = exogs_bj.copy() # ====》列表 ：除了北京内生变量以外其它所有变量不变：外生变量或因子
df_ev = data_ev.copy() #数据源 ： 数据框
preds =  pd.DataFrame(index =data_ev.index)  # 空数据框
conver_times = 0
i = 0
while True :

    i += 1
    print("-----------fold n ----", i ," -------------------" )
    # 每循环一次的内生变量是否全部解出来
    exogs_fold = exogs.copy()
    for eqid in Eqs :
        exogs,df_ev,preds,Eqs = callEq(eqid)  
    if len([x for x in exogs_fold if x not in exogs]) == 0 : #外生变量没有减少
        conver_times = conver_times + 1
    print("剩余的方程数 或 内生变量 : " , len(exogs),len(Eqs))    
    if conver_times >10 :  #停止搜索解
        break     
#del exogs,df_ev,preds,Eqs

-----------fold n ---- 1  -------------------
Eq1 gdp1r_bj  ----------------   求解成功 。
Eq3 gdpr_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj', 'gdp3r_bj']
Eq4 gdp1n_bj 还不能求解 ，因还未获得外生变量:   ['gdp1d_bj']
Eq5 gdp2_indan_bj 还不能求解 ，因还未获得外生变量:   ['gdp2_indar_bj']
Eq6 gdp2_bindan_bj 还不能求解 ，因还未获得外生变量:   ['gdp2_bindar_bj']
Eq7 gdp2n_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj', 'gdp2d_bj']
Eq8 gdp2r_bj2 还不能求解 ，因还未获得外生变量:   ['gdp2_indar_bj', 'gdp2_bindar_bj']
Eq9 gdp2r_bj1 还不能求解 ，因还未获得外生变量:   ['gdp_tecn_bj', 'gdp2d_bj']
Eq10 gdp2r_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj1', 'gdp2r_bj2']
Eq11 gdp3n_bj 还不能求解 ，因还未获得外生变量:   ['gdp3r_bj', 'gdp3d_bj']
Eq12 gdp2_indar_bj 还不能求解 ，因还未获得外生变量:   ['consr_bj', 'investr_bj', 'exusd_bj']
Eq13 gdp2_bindar_bj 还不能求解 ，因还未获得外生变量:   ['investr_bj']
Eq14 gdp3r_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj', 'exusd_bj']
Eq15 gdp3r_bj 还不能求解 ，因还未获得外生变量:   ['con_pr_bj', 'con_gr_bj']
Eq16 con_gr_bj 还不能求解 ，因还未获得外生变量:   ['con_gn_bj']
Eq17 con_pr_bj 还不能求解 ，因还未获得外生变量:   ['con_pur_bj', 'con_prr_bj']
Eq18 con_fn_bj 还不能求解 

Eq94 gdp_tecn_bj  ----------------   求解成功 。
Eq96 gdp_mosn_bj 还不能求解 ，因还未获得外生变量:   ['gdp3_infn_bj', 'gdp3_renn_bj', 'gdp3_sicn_bj']
Eq97 gdp_momn_bj  ----------------   求解成功 。
剩余的方程数 或 内生变量 :  86 86
-----------fold n ---- 3  -------------------
Eq2 gdpn_bj 还不能求解 ，因还未获得外生变量:   ['gdp1n_bj', 'gdp2n_bj', 'gdp3n_bj']
Eq3 gdpr_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj', 'gdp3r_bj']
Eq4 gdp1n_bj 还不能求解 ，因还未获得外生变量:   ['gdp1d_bj']
Eq5 gdp2_indan_bj 还不能求解 ，因还未获得外生变量:   ['gdp2_indar_bj']
Eq6 gdp2_bindan_bj 还不能求解 ，因还未获得外生变量:   ['gdp2_bindar_bj']
Eq7 gdp2n_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj', 'gdp2d_bj']
Eq8 gdp2r_bj2 还不能求解 ，因还未获得外生变量:   ['gdp2_indar_bj', 'gdp2_bindar_bj']
Eq9 gdp2r_bj1 还不能求解 ，因还未获得外生变量:   ['gdp2d_bj']
Eq10 gdp2r_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj1', 'gdp2r_bj2']
Eq11 gdp3n_bj 还不能求解 ，因还未获得外生变量:   ['gdp3r_bj', 'gdp3d_bj']
Eq12 gdp2_indar_bj 还不能求解 ，因还未获得外生变量:   ['consr_bj', 'investr_bj']
Eq13 gdp2_bindar_bj 还不能求解 ，因还未获得外生变量:   ['investr_bj']
Eq14 gdp3r_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj']
Eq15 gdp

Eq8 gdp2r_bj2 还不能求解 ，因还未获得外生变量:   ['gdp2_indar_bj', 'gdp2_bindar_bj']
Eq9 gdp2r_bj1 还不能求解 ，因还未获得外生变量:   ['gdp2d_bj']
Eq10 gdp2r_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj1', 'gdp2r_bj2']
Eq11 gdp3n_bj 还不能求解 ，因还未获得外生变量:   ['gdp3r_bj', 'gdp3d_bj']
Eq12 gdp2_indar_bj 还不能求解 ，因还未获得外生变量:   ['consr_bj', 'investr_bj']
Eq13 gdp2_bindar_bj 还不能求解 ，因还未获得外生变量:   ['investr_bj']
Eq14 gdp3r_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj']
Eq15 gdp3r_bj 还不能求解 ，因还未获得外生变量:   ['con_pr_bj', 'con_gr_bj']
Eq16 con_gr_bj 还不能求解 ，因还未获得外生变量:   ['con_gn_bj']
Eq17 con_pr_bj 还不能求解 ，因还未获得外生变量:   ['con_pur_bj', 'con_prr_bj']
Eq18 con_fn_bj 还不能求解 ，因还未获得外生变量:   ['con_pn_bj', 'con_gn_bj']
Eq19 con_pn_bj 还不能求解 ，因还未获得外生变量:   ['con_pun_bj', 'con_prn_bj']
Eq20 con_pun_bj 还不能求解 ，因还未获得外生变量:   ['con_pur_bj']
Eq21 con_prn_bj 还不能求解 ，因还未获得外生变量:   ['con_prr_bj']
Eq22 con_gn_bj 还不能求解 ，因还未获得外生变量:   ['fen_bj']
Eq23 investn_bj 还不能求解 ，因还未获得外生变量:   ['investr_bj', 'faipi_bj']
Eq24 inv_fon_bj 还不能求解 ，因还未获得外生变量:   ['inv_for_bj']
Eq26 consr_bj 还不能求解 ，因还未获得外生变量:  

Eq67 emp2_bj 还不能求解 ，因还未获得外生变量:   ['gdpr_bj']
Eq68 emp3_bj 还不能求解 ，因还未获得外生变量:   ['gdp3r_bj']
Eq69 totpop_bj 还不能求解 ，因还未获得外生变量:   ['gdpr_bj']
Eq71 rpi_bj 还不能求解 ，因还未获得外生变量:   ['cpi_bj']
Eq72 cpi_bj 还不能求解 ，因还未获得外生变量:   ['gdpd_bj']
Eq73 gdp1d_bj 还不能求解 ，因还未获得外生变量:   ['rpi_bj']
Eq74 gdp2d_bj 还不能求解 ，因还未获得外生变量:   ['ppi_bj', 'faipi_bj']
Eq75 gdp3d_bj 还不能求解 ，因还未获得外生变量:   ['gdp2d_bj']
Eq76 gdpd_bj 还不能求解 ，因还未获得外生变量:   ['gdp1d_bj', 'gdp2d_bj', 'gdp3d_bj']
Eq77 ppi_bj 还不能求解 ，因还未获得外生变量:   ['rpi_bj']
Eq78 faipi_bj 还不能求解 ，因还未获得外生变量:   ['ppi_bj']
Eq81 gdp3_wrar_bj 还不能求解 ，因还未获得外生变量:   ['gdp3_wran_bj']
Eq82 gdp3_wran_bj 还不能求解 ，因还未获得外生变量:   ['consn_bj']
Eq83 gdp3_fiar_bj 还不能求解 ，因还未获得外生变量:   ['gdp3_fian_bj']
Eq84 gdp3_fian_bj 还不能求解 ，因还未获得外生变量:   ['depn_bj']
Eq92 gdp3_sicn_bj 还不能求解 ，因还未获得外生变量:   ['gdp3_sicn_bj']
Eq96 gdp_mosn_bj 还不能求解 ，因还未获得外生变量:   ['gdp3_sicn_bj']
剩余的方程数 或 内生变量 :  79 79
-----------fold n ---- 11  -------------------
Eq2 gdpn_bj 还不能求解 ，因还未获得外生变量:   ['gdp1n_bj', 'gdp2n_bj', 'gdp3n_bj']
Eq3 gdpr_

Eq68 emp3_bj 还不能求解 ，因还未获得外生变量:   ['gdp3r_bj']
Eq69 totpop_bj 还不能求解 ，因还未获得外生变量:   ['gdpr_bj']
Eq71 rpi_bj 还不能求解 ，因还未获得外生变量:   ['cpi_bj']
Eq72 cpi_bj 还不能求解 ，因还未获得外生变量:   ['gdpd_bj']
Eq73 gdp1d_bj 还不能求解 ，因还未获得外生变量:   ['rpi_bj']
Eq74 gdp2d_bj 还不能求解 ，因还未获得外生变量:   ['ppi_bj', 'faipi_bj']
Eq75 gdp3d_bj 还不能求解 ，因还未获得外生变量:   ['gdp2d_bj']
Eq76 gdpd_bj 还不能求解 ，因还未获得外生变量:   ['gdp1d_bj', 'gdp2d_bj', 'gdp3d_bj']
Eq77 ppi_bj 还不能求解 ，因还未获得外生变量:   ['rpi_bj']
Eq78 faipi_bj 还不能求解 ，因还未获得外生变量:   ['ppi_bj']
Eq81 gdp3_wrar_bj 还不能求解 ，因还未获得外生变量:   ['gdp3_wran_bj']
Eq82 gdp3_wran_bj 还不能求解 ，因还未获得外生变量:   ['consn_bj']
Eq83 gdp3_fiar_bj 还不能求解 ，因还未获得外生变量:   ['gdp3_fian_bj']
Eq84 gdp3_fian_bj 还不能求解 ，因还未获得外生变量:   ['depn_bj']
Eq92 gdp3_sicn_bj 还不能求解 ，因还未获得外生变量:   ['gdp3_sicn_bj']
Eq96 gdp_mosn_bj 还不能求解 ，因还未获得外生变量:   ['gdp3_sicn_bj']
剩余的方程数 或 内生变量 :  79 79
-----------fold n ---- 17  -------------------
Eq2 gdpn_bj 还不能求解 ，因还未获得外生变量:   ['gdp1n_bj', 'gdp2n_bj', 'gdp3n_bj']
Eq3 gdpr_bj 还不能求解 ，因还未获得外生变量:   ['gdp2r_bj', 'gdp3r_bj

=======》 咋还变多了？因变量名称改的不一致？？？

## 方程求解二： IV3LS method GLS

GLS estimator for the linear regression model with ARMA(p, q) disturbances

### 定义方程


In [22]:
Eq2 = "gdpn_bj ~ gdp1n_bj + gdp2n_bj + gdp3n_bj "
Eq3 = " gdpr_bj ~ gdp1r_bj + gdp2r_bj + gdp3r_bj"
Eq4 = "gdp1n_bj ~ gdp1r_bj * gdp1d_bj / 100"
Eq5 = " gdp2_bindan_bj ~ gdp2_bindar_bj * gdp2_bindad_bj / 100"
Eq6 = "  gdp2_bindan_bj ~ gdp2_bindar_bj * gdp2_bindad_bj / 100"
Eq7 = "gdp2n_bj ~ gdp2r_bj * gdp2d_bj / 100"
Eq8 = "gdp2r_bj2 ~ gdp2_indar_bj + gdp2_bindar_bj + gdp2r_erro_bj"
Eq10 = "gdp2r_bj ~ (gdp2r_bj1 + gdp2r_bj2) / 2"
Eq11 = " gdp3n_bj ~ gdp3r_bj * gdp3d_bj / 100"
Eq15 = "con_fr_bj ~ con_pr_bj + con_gr_bj"
Eq16 = "con_gr_bj ~ con_gn_bj / pcon_gd_bj * 100"
Eq17 = " con_pr_bj ~ con_pur_bj + con_prr_bj"
Eq18 = "con_fn_bj ~ con_pn_bj + con_gn_bj"
Eq19 = "con_pn_bj ~ con_pun_bj + con_prn_bj"
Eq20 = "con_pun_bj ~ con_pur_bj * pcon_pud_bj / 100"
Eq21 = "con_prn_bj ~ con_prr_bj * pcon_prd_bj / 100"
Eq23 = "investn_bj ~ investr_bj * faipi_bj / 100"
Eq24 = "inv_fon_bj ~ inv_for_bj * pinv_fod_bj / 100"
Eq25 = "inv_chn_bj ~ inv_chr_bj * pinv_chd_bj / 100"
Eq26 = "consr_bj ~ consn_bj / rpi_bj * 100"
Eq27 = "inv_for_bj ~ inv_fir_bj + inv_chr_bj"
Eq28 = "inv_fin_bj ~ inv_fir_bj * pinv_fid_bj / 100"
Eq29 = "gdper_bj ~ con_fr_bj + inv_for_bj + nex_gsr_bj"
Eq30 = "gdpen_bj ~ con_fn_bj + inv_fon_bj + nex_gsn_bj"
Eq31 = "nex_gsn_bj ~ nex_gsr_bj * pnex_gsd_bj / 100"
Eq40 = "yht_n_bj ~ yht_rn_bj + yht_un_bj"
Eq41 = "yh_un_bj ~ yht_un_bj / popu_bj"
Eq42 = "yh_rn_bj ~ yht_rn_bj / popr_bj"
Eq43 = "yhn_bj ~ yht_n_bj / totpop_bj"
Eq44 = "conet_un_bj ~ cone_un_bj * popu_bj"
Eq45 = "conet_rn_bj ~ cone_rn_bj * popr_bj"
Eq53 = "taxn_bj ~ (taxn_bj1 + taxn_bj2) / 2"
Eq59 = "fen_bj ~ (fen_bj1 + fen_bj2)/2"
Eq64 = "emp1_bj ~ totemp_bj - emp2_bj - emp3_bj"
Eq65 = "popr_bj ~ totpop_bj - popu_bj"
Eq79 = "gdp3_tspar_bj ~ gdp3_tspan_bj / pgdp3_tspad_bj * 100"
Eq81 = "gdp3_wrar_bj ~ gdp3_wran_bj / pgdp3_wrad_bj * 100"
Eq83 = "gdp3_fiar_bj ~ gdp3_fian_bj / pgdp3_fiad_bj * 100"
Eq85 = "gdp3_rear_bj ~ gdp3_rean_bj / pgdp3_read_bj * 100"
Eq87 = "gdp3_infr_bj ~ gdp3_infn_bj / pgdp3_infd_bj * 100"
Eq89 = "gdp3_renr_bj ~ gdp3_renn_bj / pgdp3_rend_bj * 100"
Eq91 = "gdp3_sicr_bj ~ gdp3_sicn_bj / pgdp3_sicd_bj * 100"

### 行为方程
 行为方程 ID：
 
[x for x in Eqlist if x not in list(eq_disc[eq_disc["type"] == "定义方程"]["ID"])]


- 定义式写入行为方程，则在使用时，将实时计算。如果未写入则可能自变量已经发生变化，但是内生变量却未进行计算
- 避免同一个行为方程式中有多个等式方程

In [23]:
def diff(var , n ,m ) :
    return var.shift(n) -var.shift(m)

def gdp2gdp3tax( col1,col2,col3):  #Eq46
    return col1 + col2 - col3
#例子 ： diff("gdp1r_bj",0,1),diff("gdp1r_bj",1,2)

def fen_sum(col1,col2,col3,col4,col5,col6) : #Eq58
    return col1 + col2 + col3 + col4 + col5 + col6

def lagsum(var , n ,m) :
    return var.shift(n) + var.shift(m)
def pch(var):
    return  (var - var.shift(1) )/  var.shift(1) 
def prd (var1,var2)  :                   #Eq47
    return var1*var2 /100

In [24]:
diff( df_ev["yht_n_bj"] ,0,1)

1996            NaN
1997      38.007440
1998      82.962498
1999      94.566928
2000     233.275734
2001     181.822332
2002     167.839425
2003     230.873304
2004     301.724860
2005     451.211447
2006     497.304181
2007     506.949956
2008     719.837216
2009     626.279298
2010     817.143948
2011     952.493852
2012     916.241585
2013    1000.537092
2014     945.350395
2015     831.122780
2016     893.400000
2017    1029.630000
2018    1111.285800
2019            NaN
2020            NaN
Name: yht_n_bj, dtype: float64

In [25]:
# 定义方程的变量或表达式
defvar =  list(eq_disc[eq_disc["type"] == "定义方程"]["y"])
# test 表达式中的自变量是否可以用定义方程式替换
vars =["lse_bj","d_tta_bj"] 
done =[]
nondone = [x for x in  defvar if x not in done]
for var in vars :
    print (var ," : ","有定义方程" if var in defvar else "没有定义方程   ","已写入其他行为方程" if var in nondone else "还未没写入其他行为方程 。","\n      如果不是_bj ，_tj ，_hb ,_cn结尾 : 一般都是是衍生变量 ，需要加表达式或函数.\n " )


lse_bj  :  没有定义方程    还未没写入其他行为方程 。 
      如果不是_bj ，_tj ，_hb ,_cn结尾 : 一般都是是衍生变量 ，需要加表达式或函数.
 
d_tta_bj  :  没有定义方程    还未没写入其他行为方程 。 
      如果不是_bj ，_tj ，_hb ,_cn结尾 : 一般都是是衍生变量 ，需要加表达式或函数.
 


- 尝试 用 ** 替代 * , 又换成了 I（）
- 表达式中的变量 , 就是数据框中的字段 ，不用写数据框 ；
- Eq37 ： 有变动， 因“满秩” 原因  ，删除了 + d_consn_bj

<font color = "blue" size=5 > 原方程 </font>


In [26]:
Eq1 =   " gdp1r_bj ~ cul_area_bj  + gdp1r_bj.shift(1) + d_gdp1r_bj " 
Eq9 =   " gdp2r_bj1 ~ gdp_tecn_bj / gdp2d_bj  + gdp2r_bj1.shift(1) + d_gdp2r_bj_1 " 
Eq12 =  " np.log(gdp2_indar_bj) ~ [np.log(consr_bj) ~ np.log(consn_bj /rpi_bj) ]  + np.log(investr_bj) + np.log(exusd_bj * exrate_cn) + d_gdp2_indar_bj " 
Eq13 =  " np.log(gdp2_bindar_bj) ~ np.log(investr_bj)"    # SARMAX ,AR(2) 
Eq14 =  " np.log(gdp3r_bj) ~ [np.log(gdp2r_bj) ~ np.log((gdp2r_bj1 + gdp2r_bj2)/2)]  + np.log(exusd_bj * exrate_cn) +  np.log(gdp3r_bj.shift(1)) "
Eq22 =  " con_gn_bj ~ [fen_bj ~ fen_bj1 + fen_bj2 ] " 
Eq32 =  " con_pur_bj ~ (yht_un_bj / pcon_pud_bj) ** 100" 
Eq33 =  " con_prr_bj ~ yht_rn_bj / pcon_prd_bj **100  + diff(con_prr_bj ,1,2) " 
Eq34 =  " investr_bj ~ (loann_bj/faipi_bj) ** 100 + d_investr_bj" 
Eq35 =  " inv_fir_bj ~ investr_bj + inv_fir_bj.shift(1)" 
Eq36 =  " nex_gsr_bj ~ [gdpr_bj ~ gdp1r_bj + gdp2r_bj + gdp3r_bj ] + nex_gsr_bj.shift(1) + d_nex_gsr_bj" 
Eq37 =  " consn_bj ~ [diff( yht_n_bj,0,1) ~ diff(yht_rn_bj,0,1) + diff(yht_un_bj ,0,1) ] +consn_bj.shift(1)"  #   + d_consn_bj  ，[yht_n_bj ~ yht_rn_bj + yht_un_bj]
Eq38 =  " exusd_bj ~ exusd_ch +exusd_bj.shift(1) + diff(exusd_bj,1,2) + d_exusd_bj"  #diff(exusd_bj,1,2)
Eq39 =  " np.log(imusd_bj) ~ np.log(imusd_bj.shift(1)) + np.log(exusd_bj) + d_imusd_bj"
Eq46 =  " yht_un_bj ~  [gdp2gdp3tax(gdp2n_bj,gdp3n_bj ,taxn_bj) ~ I((gdp2r_bj * gdp2d_bj) / 100) +I( (gdp3r_bj * gdp3d_bj) / 100) - I( (taxn_bj1 + taxn_bj2) / 2)] +  yht_un_bj.shift(1)"

# gdp2gdp3tax([gdp2n_bj ~ gdp2r_bj * gdp2d_bj / 100 ] , [gdp3n_bj ~ gdp3r_bj * gdp3d_bj / 100] , [taxn_bj ~ (taxn_bj1 + taxn_bj2) / 2]) + yht_un_bj.shift(1)" 

Eq47 =  " yht_rn_bj ~ prd(gdp1r_bj , gdp1d_bj) + prd( gdp3r_bj , gdp3d_bj) + yht_rn_bj.shift(1)" 
#[gdp1n_bj ~ gdp1r_bj * gdp1d_bj / 100] + [gdp3n_bj ~ gdp3r_bj * gdp3d_bj / 100]

Eq48 =  " cone_un_bj ~ diff([ yh_un_bj ~ yht_un_bj / popu_bj] , 0, 1) + cone_un_bj.shift(1)" 
Eq49 =  " cone_rn_bj ~ [yh_rn_bj ~ yht_rn_bj / popr_bj]+ d_cone_rn_bj" 
Eq50 =  " frn_bj ~ [taxn_bj ~ (taxn_bj1 + taxn_bj2) / 2] + frn_bj.shift(1)" 
Eq51 =  " np.log(taxn_bj1) ~ np.log(tax_vadn_bj+tax_cinn_bj+tax_pinn_bj) + d_taxn_bj_1"  # SARMAX ,AR(1)
Eq52 =  " taxn_bj2 ~ [gdpn_bj ~ gdp1n_bj + gdp2n_bj + gdp3n_bj]+ taxn_bj2.shift(1)"
Eq54 =  " np.log(tax_vadn_bj) ~ np.log([gdpn_bj ~ gdp1n_bj + gdp2n_bj + gdp3n_bj]) + np.log(tax_vadn_bj.shift(1)) + d_tax_vadn_bj" 
Eq55 =  " tax_cinn_bj ~ [gdpn_bj ~ gdp1n_bj + gdp2n_bj + gdp3n_bj] + tax_cinn_bj.shift(1)" 
Eq56 =  " tax_pinn_bj ~ [yht_n_bj ~ yht_rn_bj + yht_un_bj]+ tax_pinn_bj.shift(1)" 
Eq57 =  " fen_bj1 ~ frn_bj"   # SARMAX  ,AR(1)
Eq58 =  " fen_bj2 ~ fen_sum(fe_edun_bj,fe_tecn_bj,fe_culn_bj,fe_ssen_bj,fe_medn_bj,fe_eepn_bj) + d_fen_bj_2" 
Eq60 =  " np.log(depn_bj) ~ np.log(dep_san_bj) + np.log(depn_bj.shift(1)) + d_depn_bj" 
Eq61 =  " np.log(dep_san_bj) ~ np.log([yht_n_bj ~ yht_rn_bj + yht_un_bj]) + np.log(dep_san_bj.shift(1)) + d_dep_san_bj" 
Eq62 =  " np.log(loan_ln_bj) ~ np.log(depn_bj) + np.log(loan_ln_bj.shift(1))" 
Eq63 =  " loann_bj ~ lagsum(depn_bj,0,1) + loann_bj.shift(1)  + intratel_cn + d_loann_bj" 
Eq66 =  " totemp_bj ~ diff([gdpr_bj ~ gdp1r_bj + gdp2r_bj + gdp3r_bj ],0,1) + totemp_bj.shift(1)  + d_totemp_bj" 
Eq67 =  " emp2_bj ~ [gdpr_bj ~ gdp1r_bj + gdp2r_bj + gdp3r_bj ] + pch([gdpr_bj ~ gdp1r_bj + gdp2r_bj + gdp3r_bj ])  + pch(emp2_bj) + emp2_bj.shift(1)" 
Eq68 =  " emp3_bj ~ pch(gdp3r_bj) + emp3_bj.shift(1)  + d_emp3_bj" 
Eq69 =  " totpop_bj ~ diff([gdpr_bj ~ gdp1r_bj + gdp2r_bj + gdp3r_bj ],0,1) + totpop_bj.shift(1)  + d_totpop_bj" 
Eq70 =  " popu_bj ~ popu_bj.shift(1)","d_popu_bj"   # sarmax ,AR(1)
Eq71 =  " rpi_bj ~ diff(cpi_bj,0,1) + rpi_bj.shift(1)  + d_rpi_bj"
Eq72 =  " cpi_bj ~ cpi_bj.shift(1) + diff(gdpd_bj,0,1) + d_cpi_bj"  # sarmax ,AR(1) 
Eq73 =  " gdp1d_bj ~ diff(rpi_bj,0,1) + gdp1d_bj.shift(1)" 
Eq74 =  " np.log(gdp2d_bj) ~ np.log(ppi_bj) + np.log(faipi_bj) + d_gdp2d_bj" 
Eq75 =  " gdp3d_bj ~ gdp2d_bj + gdp3d_bj.shift(1)" 
Eq76 =  " gdpd_bj ~ gdp1d_bj + gdp2d_bj + gdp3d_bj"
Eq77 =  " np.log(ppi_bj) ~ diff(rpi_bj,0,1) + np.log(ppi_bj_shift(1))  + d_ppi_bj"
Eq78 =  " np.log(faipi_bj) ~ diff(ppi_bj,0,1) + np.log(faipi_bj.shift(1))  + d_faipi_bj" 
Eq80 =  " gdp3_tspan_bj~ gdp3_tratp_bj + gdp3_tragt_bj  + gdp3_trapt_bj" 
Eq82 =  " gdp3_wran_bj~ diff(consn_bj,0,1) + gdp3_wran_bj.shift(1)" 
Eq84 =  " gdp3_fian_bj~ depn_bj + gdp3_fian_bj.shift(1)" 
Eq86 =  " gdp3_rean_bj~ inv_rean_bj + gdp3_rean_bj.shift(1) + d_gdp3_rean_bj" 
Eq88 =  " gdp3_infn_bj~ gdp_tran_bj" 
Eq90 =  " gdp3_renn_bj~ gdp_momn_bj" 
Eq92 =  " gdp3_sicn_bj~ gdp_momn_bj + gdp3_sicn_bj.shift(1)" 
Eq93 =  " gdp_tran_bj~ gdp_tecn_bj + d_gdp_tran_bj" 
Eq94 =  " gdp_tecn_bj ~ lse_bj + diff(tta_bj,0,1) + gdp_tecn_bj.shift(1) + d_gdp_tecn_bj"
Eq95 =  " tta_f_bj~ tta_bj"
Eq96 =  " gdp_mosn_bj ~ gdp3_infn_bj + gdp3_renn_bj + gdp3_sicn_bj + d_gdp_mosn_bj" 
Eq97 =  " gdp_momn_bj ~ diff(gdp_tecn_bj,0,1) + gdp_momn_bj.shift(1)" 
Eq98 =  " gdp_infn_bj ~ gdp_tecn_bj + gdp_infn_bj.shift(1) + d_gdp_infn_bj" 
Eq99 =  " tta_bj ~ lse_bj + diff(tta_bj,0,1) + d_tta_bj" 

In [175]:
Eq1 =   " gdp1r_bj ~ 1 + cul_area_bj  + gdp1r_bj.shift(1) + d_gdp1r_bj " 
Eq9 =   " gdp2r_bj1 ~  + gdp_tecn_bj / gdp2d_bj  + gdp2r_bj1.shift(1) + d_gdp2r_bj_1 " 
Eq12 =  " np.log(gdp2_indar_bj) ~ 1 +  [np.log(consr_bj) ~ np.log(consn_bj /rpi_bj) ]  + np.log(investr_bj) + np.log(exusd_bj * exrate_cn) + d_gdp2_indar_bj " 
Eq13 =  " np.log(gdp2_bindar_bj) ~  1 + np.log(investr_bj)"    # SARMAX ,AR(2) 
Eq14 =  " np.log(gdp3r_bj) ~ 1 + [np.log(gdp2r_bj) ~ np.log((gdp2r_bj1 + gdp2r_bj2)/2)]  + np.log(exusd_bj * exrate_cn) +  np.log(gdp3r_bj.shift(1)) "
Eq22 =  " con_gn_bj ~  1 + [fen_bj ~ fen_bj1 + fen_bj2 ] " 
Eq32 =  " con_pur_bj ~ 1 + (yht_un_bj / pcon_pud_bj) ** 100" 
Eq33 =  " con_prr_bj ~ yht_rn_bj / pcon_prd_bj **100  + diff(con_prr_bj ,1,2) " 
Eq34 =  " investr_bj ~ (loann_bj/faipi_bj) ** 100 + d_investr_bj" 
Eq35 =  " inv_fir_bj ~ investr_bj + inv_fir_bj.shift(1)" 
Eq36 =  " nex_gsr_bj ~ [gdpr_bj ~ gdp1r_bj + gdp2r_bj + gdp3r_bj ] + nex_gsr_bj.shift(1) + d_nex_gsr_bj" 
Eq37 =  " consn_bj ~ [diff( yht_n_bj,0,1) ~ diff(yht_rn_bj,0,1) + diff(yht_un_bj ,0,1) ] +consn_bj.shift(1)"  #   + d_consn_bj  ，[yht_n_bj ~ yht_rn_bj + yht_un_bj]
Eq38 =  " exusd_bj ~ exusd_ch +exusd_bj.shift(1) + diff(exusd_bj,1,2) + d_exusd_bj"  #diff(exusd_bj,1,2)
Eq39 =  " np.log(imusd_bj) ~ np.log(imusd_bj.shift(1)) + np.log(exusd_bj) + d_imusd_bj"
Eq46 =  " yht_un_bj ~  [gdp2gdp3tax(gdp2n_bj,gdp3n_bj ,taxn_bj) ~ I((gdp2r_bj * gdp2d_bj) / 100) +I( (gdp3r_bj * gdp3d_bj) / 100) - I( (taxn_bj1 + taxn_bj2) / 2)] +  yht_un_bj.shift(1)"
# gdp2gdp3tax([gdp2n_bj ~ gdp2r_bj * gdp2d_bj / 100 ] , [gdp3n_bj ~ gdp3r_bj * gdp3d_bj / 100] , [taxn_bj ~ (taxn_bj1 + taxn_bj2) / 2]) + yht_un_bj.shift(1)" 
Eq47 =  " yht_rn_bj ~ prd(gdp1r_bj , gdp1d_bj) + prd( gdp3r_bj , gdp3d_bj) + yht_rn_bj.shift(1)" 
#[gdp1n_bj ~ gdp1r_bj * gdp1d_bj / 100] + [gdp3n_bj ~ gdp3r_bj * gdp3d_bj / 100]
Eq48 =  " cone_un_bj ~ [diff( yh_un_bj,0,1) ~ diff(yht_un_bj / popu_bj , 0, 1) ]+ cone_un_bj.shift(1)" 
# diff([ yh_un_bj ~ yht_un_bj / popu_bj] , 0, 1)
Eq49 =  " cone_rn_bj ~ [yh_rn_bj ~ yht_rn_bj / popr_bj]+ d_cone_rn_bj" 
Eq50 =  " frn_bj ~ [taxn_bj ~ I((taxn_bj1 + taxn_bj2) / 2)] + frn_bj.shift(1)" 
Eq51 =  " np.log(taxn_bj1) ~ np.log(tax_vadn_bj+tax_cinn_bj+tax_pinn_bj) + d_taxn_bj_1"  # SARMAX ,AR(1)
Eq52 =  " taxn_bj2 ~ [gdpn_bj ~ gdp1n_bj + gdp2n_bj + gdp3n_bj]+ taxn_bj2.shift(1)"
Eq54 =  " np.log(tax_vadn_bj) ~ [np.log(gdpn_bj) ~ np.log(gdp1n_bj + gdp2n_bj + gdp3n_bj)] + np.log(tax_vadn_bj.shift(1)) + d_tax_vadn_bj" 
Eq55 =  " tax_cinn_bj ~ [gdpn_bj ~ gdp1n_bj + gdp2n_bj + gdp3n_bj] + tax_cinn_bj.shift(1)" 
Eq56 =  " tax_pinn_bj ~ [yht_n_bj ~ yht_rn_bj + yht_un_bj]+ tax_pinn_bj.shift(1)" 
Eq57 =  " fen_bj1 ~ frn_bj"   # SARMAX  ,AR(1)
Eq58 =  " fen_bj2 ~ fen_sum(fe_edun_bj,fe_tecn_bj,fe_culn_bj,fe_ssen_bj,fe_medn_bj,fe_eepn_bj) + d_fen_bj_2" 
Eq60 =  " np.log(depn_bj) ~ np.log(dep_san_bj) + np.log(depn_bj.shift(1)) + d_depn_bj" 
Eq61 =  " np.log(dep_san_bj) ~ [np.log(yht_n_bj) ~ np.log ( yht_rn_bj + yht_un_bj)] + np.log(dep_san_bj.shift(1)) + d_dep_san_bj" 
Eq62 =  " np.log(loan_ln_bj) ~ np.log(depn_bj) + np.log(loan_ln_bj.shift(1))" 
Eq63 =  " loann_bj ~ lagsum(depn_bj,0,1) + loann_bj.shift(1)  + intratel_cn + d_loann_bj" 
Eq66 =  " totemp_bj ~ [diff(gdpr_bj,0,1) ~ diff(gdp1r_bj + gdp2r_bj + gdp3r_bj ,0,1)] + totemp_bj.shift(1)  + d_totemp_bj" 
Eq67 =  " emp2_bj ~ [gdpr_bj ~ gdp1r_bj + gdp2r_bj + gdp3r_bj ] + pch( gdp1r_bj + gdp2r_bj + gdp3r_bj)  + pch(emp2_bj) + emp2_bj.shift(1)" 
#Eq67 : 少一个中括号 pch([gdpr_bj ~ gdp1r_bj + gdp2r_bj + gdp3r_bj ])
Eq68 =  " emp3_bj ~   pch(gdp3r_bj) + emp3_bj.shift(1) "
# Eq68 : 少一个 d_emp3_bj ，因满秩
Eq69 =  " totpop_bj ~ [diff(gdpr_bj,0,1) ~ diff(gdp1r_bj + gdp2r_bj + gdp3r_bj,0,1) ] + totpop_bj.shift(1)  + d_totpop_bj" 
Eq70 =  " popu_bj ~ popu_bj.shift(1) + d_popu_bj"   # sarmax ,AR(1)
Eq71 =  " rpi_bj ~ diff(cpi_bj,0,1) + rpi_bj.shift(1)  + d_rpi_bj"
Eq72 =  " cpi_bj ~ cpi_bj.shift(1) + diff(gdpd_bj,0,1) + d_cpi_bj"  # sarmax ,AR(1) 
Eq73 =  " gdp1d_bj ~ diff(rpi_bj,0,1) + gdp1d_bj.shift(1)" 
Eq74 =  " np.log(gdp2d_bj) ~ np.log(ppi_bj) + np.log(faipi_bj) + d_gdp2d_bj" 
Eq75 =  " gdp3d_bj ~ gdp2d_bj + gdp3d_bj.shift(1)" 
Eq76 =  " gdpd_bj ~ gdp1d_bj + gdp2d_bj + gdp3d_bj"
Eq77 =  " np.log(ppi_bj) ~ diff(rpi_bj,0,1) + np.log(ppi_bj.shift(1))  + d_ppi_bj"
Eq78 =  " np.log(faipi_bj) ~ diff(ppi_bj,0,1) + np.log(faipi_bj.shift(1))  + d_faipi_bj" 
Eq80 =  " gdp3_tspan_bj~ gdp3_tratp_bj + gdp3_tragt_bj  + gdp3_trapt_bj" 
Eq82 =  " gdp3_wran_bj~ diff(consn_bj,0,1) + gdp3_wran_bj.shift(1)" 
Eq84 =  " gdp3_fian_bj~ depn_bj + gdp3_fian_bj.shift(1)" 
Eq86 =  " gdp3_rean_bj~ inv_rean_bj + gdp3_rean_bj.shift(1) + d_gdp3_rean_bj" 

Eq88 =  " gdp3_infn_bj~ 1 + gdp_tran_bj" 
Eq90 =  " gdp3_renn_bj~ 1 + gdp_momn_bj" 
Eq92 =  " gdp3_sicn_bj~ gdp_momn_bj + gdp3_sicn_bj.shift(1)" 
Eq93 =  " gdp_tran_bj~ gdp_tecn_bj + d_gdp_tran_bj" 
Eq94 =  " gdp_tecn_bj ~ lse_bj + diff(tta_bj,0,1) + gdp_tecn_bj.shift(1) + d_gdp_tecn_bj"
Eq95 =  " tta_f_bj~ tta_bj"
Eq96 =  " gdp_mosn_bj ~ gdp3_infn_bj + gdp3_renn_bj + gdp3_sicn_bj + d_gdp_mosn_bj" 
Eq97 =  " gdp_momn_bj ~ diff(gdp_tecn_bj,0,1) + gdp_momn_bj.shift(1)" 
Eq98 =  " gdp_infn_bj ~ gdp_tecn_bj + gdp_infn_bj.shift(1) + d_gdp_infn_bj" 
Eq99 =  " tta_bj ~ lse_bj + diff(tta_bj,0,1) + d_tta_bj" 

In [179]:
df_ev[["gdp3_infn_bj" , "gdp_tran_bj"]]

,gdp3_infn_bj,gdp_tran_bj
1996,NaN,NaN
1997,NaN,NaN
1998,NaN,NaN
1999,NaN,NaN
2000,164.4,NaN
2001,210.1,NaN
2002,278.6,NaN
2003,378.0,NaN
2004,449.6,NaN
2005,586.6,NaN


In [176]:
system = {'Eq1' : Eq1 ,
 'Eq9' : Eq9 ,         
 'Eq12' : Eq12 ,
 'Eq13':  Eq13    ,
 'Eq14':  Eq14    ,
 'Eq22':  Eq22    ,
 'Eq32':  Eq32    ,
 'Eq33':  Eq33    ,
 'Eq34':  Eq34    ,
 'Eq35':  Eq35    ,
 'Eq36':  Eq36    ,
 'Eq37':  Eq37    ,
 'Eq38':  Eq38    ,
 'Eq39':  Eq39    ,
 'Eq46':  Eq46    ,
 'Eq47':  Eq47    ,
 'Eq48':  Eq48    ,
 'Eq49':  Eq49    ,
 'Eq50':  Eq50    ,
 'Eq51':  Eq51    ,
 'Eq52':  Eq52    ,
 'Eq54':  Eq54    ,
 'Eq55':  Eq55    ,
 'Eq56':  Eq56    ,
 'Eq57':  Eq57    ,
 'Eq58':  Eq58    ,
 'Eq60':  Eq60    ,
 'Eq61':  Eq61    ,
 'Eq62':  Eq62    ,
 'Eq63':  Eq63    ,
 'Eq66':  Eq66    ,
 'Eq67':  Eq67    ,
 'Eq68':  Eq68    ,
 'Eq69':  Eq69    ,
 'Eq70':  Eq70    ,
 'Eq71':  Eq71    ,
 'Eq72':  Eq72    ,
 'Eq73':  Eq73    ,
 'Eq74':  Eq74    ,
 'Eq75':  Eq75    ,
 'Eq76':  Eq76    ,
 'Eq77':  Eq77    ,
 'Eq78':  Eq78    ,
 'Eq80':  Eq80    ,
 'Eq82':  Eq82    ,
 'Eq84':  Eq84    ,
 'Eq86':  Eq86    ,
 'Eq88':  Eq88    ,
 'Eq90':  Eq90    ,
 'Eq92':  Eq92    ,
 'Eq93':  Eq93    ,
 'Eq94':  Eq94    ,
 'Eq95':  Eq95    ,
 'Eq96':  Eq96    ,
 'Eq97':  Eq97    ,
 'Eq98':  Eq98    ,
 'Eq99' : Eq99 }

In [202]:
system = {'Eq1':Eq1 ,'Eq9':Eq9 , 'Eq12':Eq12 ,  'Eq13':Eq13 , 'Eq14':Eq14 , 'Eq22':Eq22 , 'Eq32':Eq32 ,  'Eq33':Eq33 ,
          'Eq34': Eq34 ,'Eq35':Eq35 ,'Eq36':Eq36 ,'Eq37': Eq37 ,'Eq38': Eq38,'Eq39': Eq39 ,'Eq46': Eq46 ,'Eq47': Eq47  ,
         'Eq48': Eq48 ,'Eq49': Eq49 , 'Eq50': Eq50 ,'Eq51': Eq51 , 'Eq52': Eq52 ,"Eq54":Eq54 ,"Eq55":Eq55 ,"Eq56":Eq56,
         "Eq57":Eq57 , "Eq58":Eq58 ,"Eq60":Eq60 , "Eq61":Eq61, "Eq62":Eq62 , "Eq63":Eq63 , "Eq66":Eq66 , "Eq67":Eq67,
         "Eq68":Eq68, "Eq69":Eq69, "Eq70":Eq70 , "Eq71":Eq71, "Eq72":Eq72, "Eq73":Eq73, "Eq74":Eq74 , "Eq75":Eq75, "Eq76":Eq76,
         "Eq77":Eq77 ,"Eq78":Eq78  ,"Eq80":Eq80 ,"Eq82":Eq82 ,"Eq84":Eq84  ,"Eq86":Eq86 ,"Eq90":Eq90 ,"Eq90":Eq90,"Eq92":Eq92 ,
         "Eq94":Eq94 ,"Eq95":Eq95 ,"Eq96":Eq96 ,"Eq97":Eq97 ,"Eq98":Eq98,"Eq99":Eq99   }
#  "Eq88":Eq88 ,"Eq93":Eq93  数据太少,"Eq94":Eq94

In [203]:
klein = IV3SLS.from_formula(system,data= df_ev)
klein_3sls = klein.fit(cov_type='unadjusted' ,method = "gls",full_cov = True )
klein_3sls.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                           System GLS Estimation Summary                           
===================================================================================
Estimator:                        GLS   Overall R-squared:                  -9936.7
No. Equations.:                    55   McElroy's R-squared:                       
No. Observations:                  10   Judge's (OLS) R-squared:            -2.3470
Date:                Sat, Jul 18 2020   Berndt's R-squared:                  1.0000
Time:                        09:21:26   Dhrymes's R-squared:             -8.285e+04
                                        Cov. Estimator:                  unadjusted
                                        Num. Constraints:                      None
                     Equation: Eq1, Dependent Variable: gdp1r_bj                     
=====================================================================================
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
Intercept            -6687.9                                                         
cul_area_bj           509.82                                                         
gdp1r_bj.shift(1)    -41.388  1.283e-09 -3.227e+10     0.0000     -41.388     -41.388
d_gdp1r_bj            156.85                                                         
                      Equation: Eq9, Dependent Variable: gdp2r_bj1                      
========================================================================================
                      Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------------
gdp_tecn_bj             -414.75                                                         
gdp_tecn_bj:gdp2d_bj     3.9700                                                         
gdp2r_bj1.shift(1)      -58.906                                                         
d_gdp2r_bj_1          2.867e+04                                                         
                   Equation: Eq12, Dependent Variable: np.log(gdp2_indar_bj)                    
================================================================================================
                              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------------------
Intercept                     1.946e+04                                                         
np.log(investr_bj)              -3978.3  6.482e-08 -6.137e+10     0.0000     -3978.3     -3978.3
np.log(exusd_bj * exrate_cn)    -1692.8                                                         
d_gdp2_indar_bj                 -102.06                                                         
np.log(consr_bj)                 4109.1   6.88e-08  5.972e+10     0.0000      4109.1      4109.1
                                                                                                
=========================
       Instruments       
-------------------------
np.log(consn_bj / rpi_bj)
                         
              Equation: Eq13, Dependent Variable: np.log(gdp2_bindar_bj)              
======================================================================================
                    Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------------
Intercept             -1612.5                                                         
np.log(investr_bj)     201.42                                                         
                      Equation: Eq14, Dependent Variable: np.log(gdp3r_bj)                      
=======================================================================

# 3LS 
- 表达式中的方程一是 ：定义方程 ，二是衍生变量 
- 同一个表达式中有多个定义方程，怎么办？系数不一样，不能单纯合并？可以分拆？加中间变量？

### 测试

结论 ：
- 目标变量是否 有np.log都会预测 原变量（不受转换函数的影响）；
- 每个函数对数据源时间段要求不一样，也没关系不受影响；
- 定义方程 要写入表达式；
- 加减法的转换可能需要写入表达式中；


In [66]:
## 行为方程 ：
df_ev = data_ev.copy()
df_ev["tta_bj_diff"] = df_ev["tta_bj"] - df_ev["tta_bj"].shift(1)
#df_ev = df_ev[(df_ev.index>2004) & (df_ev.index <2019)] #可以不用选择时间
Eq9 = "np.log(gdp2r_bj1) ~1+ gdp_tecn_bj/gdp2d_bj  + gdp2r_bj1.shift(1) + d_gdp2r_bj_1"
Eq94 = "gdp_tecn_bj ~1+  lse_bj + [np.log(tta_bj_diff) ~ np.log(tta_bj - tta_bj.shift(1))] + gdp_tecn_bj.shift(1) + d_gdp_tecn_bj"


In [67]:
system = {'Eq3':Eq3, 'Eq94':Eq94}
klein = IV3SLS.from_formula(system,data=df_ev)
klein_3sls = klein.fit(cov_type='unadjusted',method = "gls",full_cov = True )
klein_3sls.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                           System GLS Estimation Summary                           
===================================================================================
Estimator:                        GLS   Overall R-squared:                   1.0000
No. Equations.:                     2   McElroy's R-squared:                       
No. Observations:                  14   Judge's (OLS) R-squared:             0.9999
Date:                Sat, Jul 18 2020   Berndt's R-squared:                  1.0000
Time:                        03:44:53   Dhrymes's R-squared:                 0.9999
                                        Cov. Estimator:                  unadjusted
                                        Num. Constraints:                      None
                  Equation: Eq3, Dependent Variable: gdpr_bj                  
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
gdp1r_bj       1.0000  1.465e-12  6.826e+11     0.0000      1.0000      1.0000
gdp2r_bj       1.0000  2.937e-13  3.405e+12     0.0000      1.0000      1.0000
gdp3r_bj       1.0000  6.241e-14  1.602e+13     0.0000      1.0000      1.0000
                    Equation: Eq94, Dependent Variable: gdp_tecn_bj                     
========================================================================================
                      Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------------
Intercept               -249.18     139.05    -1.7920     0.0731     -521.73      23.358
lse_bj                   0.0239     0.0017     14.078     0.0000      0.0205      0.0272
gdp_tecn_bj.shift(1)     0.6332     0.0257     24.651     0.0000      0.5828      0.6835
d_gdp_tecn_bj            981.86     28.653     34.267     0.0000      925.70      1038.0
np.log(tta_bj_diff)      53.045     30.632     1.7317     0.0833     -6.9925      113.08
================================
          Instruments           
--------------------------------
np.log(tta_bj - tta_bj.shift(1))
--------------------------------

Covariance Estimator:
Homoskedastic (Unadjusted) Covariance (Debiased: False, GLS: True)
"""

In [107]:
klein_3sls.predict()

{'Eq3':               Eq3
 2005  2235.161963
 2006  2475.255385
 2007  2732.699438
 2008  2738.026411
 2009  3017.160647
 2010  3430.651567
 2011  3695.382253
 2012  3917.875604
 2013  4219.261691
 2014  4511.418352
 2015  4651.577226
 2016  4921.587865
 2017  5239.814917
 2018  5416.250922, 'Eq94':              Eq94
 2005   472.731990
 2006   598.926411
 2007   743.012775
 2008   891.838643
 2009   807.530609
 2010   834.784618
 2011  1139.420790
 2012  1187.639312
 2013  4273.525199
 2014  4682.670945
 2015  5294.214800
 2016  5745.249580
 2017  6345.082063
 2018  7057.758703}

### 拟牛顿法
- 真实值和预测值直之间的差距，可以当作每个外生变量的因子（但从现实逻辑上来看，完全没道理===》孙老师的d_*变量有可能就充当了这样的角色，但是系数却不是1

- 最终加到预测值上，使得真实值和预测值相等，优化目标就是使得这个因子越小越好，最好是零，最终可能每个等式都会加一个因子，但无所谓，在场景模拟时，会重新运算，只要思路没问题即可

和单一方程求解的区别：

1）单一方程的方法已经使得每个真实值和预测值之间的差距最小化了；

2）如果联合方程的因子个数不受约束，则单一方程和联合方程就是一样的。

3）不受约束情况下，每个场景预测值都等于真实值，

4）模拟场景重新运算后，结果仍然无差异化;

5) 一般方程求解是通过，带入实现的，但这个联立方程因涉及到AR所以有些不准 ：
   简化方法 ： 1）将所有带有AR的方程独立求解 
           2）其它方程通过完全带入的方式求解，即每个内生变量都只是外生变量的函数
           
           
6） 整个方程求解分成三步 ： 一是：简化方程模型  二是：带AR方程求解  三是：联合方程求解 

In [ ]:
def minerror(data )

    global exogs
    global df_ev
    global preds
    #print(len(exogs))
    #print(df_ev.shape)   
    #print(preds.shape)
    
    # 方程一：
    y = "gdp1r_bj"
    test1 = df_ev[["gdp1r_bj","cul_area_bj","d_gdp1r_bj"]]
    test1["gdplr_bj_s1"] = test1["gdp1r_bj"].shift(1)  ##直接写进表达式，结果有差异
    test1 = test1[(test1.index>1996) & (test1.index <2018)]
    results = sm.ols(formula = "gdp1r_bj ~ cul_area_bj  + gdplr_bj_s1 + d_gdp1r_bj",data = test1).fit()
    #------3 : 写结果
    test1[y] = results.predict()
       # 预测结果
    preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)
    
    
      
        
    if Eqid == "Eq2": #建立定义模型
            y = "gdpn_bj"
            x_list = ["gdp1n_bj","gdp2n_bj","gdp3n_bj"]
            if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
                
           # try :
                #-----2 :
                test1 = df_ev[["gdpn_bj","gdp1n_bj","gdp2n_bj","gdp3n_bj"]]
                #test1 = test1[(test1.index>1996) & (test1.index <2018)]
                #------3 : 写结果
                test1[y] = test1["gdp1n_bj"] + test1["gdp2n_bj"] + test1["gdp3n_bj"]
                # 预测结果
                preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)
                # 更新数据源相应外生变量，方便后面作为变量进行其他模型拟合
                df_ev[y] = preds[y]
                # 将该内生变量从内生变量清单中删除 ，已证明内生变量只会计算一次 ：
                exogs.remove(y)

                print(Eqid ,y,"----------------   求解成功 。")
            else:
                print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 

    if Eqid == "Eq3": #建立定义模型
            y = "gdpr_bj"
            x_list =["gdp1r_bj","gdp2r_bj","gdp3r_bj"]
            if (y in exogs) &( len([x for x in x_list if x in exogs])==0): 
           # try :                
                test1 = df_ev[["gdpr_bj","gdp1r_bj","gdp2r_bj","gdp3r_bj"]]
                #test1 = test1[(test1.index>1996) & (test1.index <2018)]                
                test1[y] = test1["gdp1r_bj"] + test1["gdp2r_bj"] + test1["gdp3r_bj"]             
                preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
                df_ev[y] = preds[y]                 
                exogs.remove(y)   
                print(Eqid ,y," ----------------   求解成功 。")
            else:
                print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
    if Eqid == "Eq4": #建立定义模型
            y = "gdp1n_bj"
            x_list =["gdp1r_bj","gdp1d_bj"]
            if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
           # try :                
                test1 = df_ev[["gdp1n_bj","gdp1r_bj","gdp1d_bj"]]
                #test1 = test1[(test1.index>1996) & (test1.index <2018)]
                test1["y"] = test1["gdp1r_bj"]*test1["gdp1d_bj"]/100            
                preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
                df_ev[y] = preds[y]                 
                exogs.remove(y)   
                print(Eqid ,y," ----------------   求解成功 。")
            else:
                print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )    
    if Eqid == "Eq5": #建立定义模型
        y = "gdp2_indan_bj"
        x_list =["gdp2_indar_bj","gdp2_indad_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp2_indan_bj","gdp2_indar_bj","gdp2_indad_bj"]]
            #test1 = test1[(test1.index>1996) & (test1.index <2018)]
            test1[y] = test1["gdp2_indar_bj"]*test1["gdp2_indad_bj"]/100           
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)   
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 

    if Eqid == "Eq6": #建立定义模型
        y = "gdp2_bindan_bj"
        x_list =["gdp2_bindar_bj","gdp2_bindad_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp2_bindan_bj","gdp2_bindar_bj","gdp2_bindad_bj"]]
            test1 = test1[(test1.index>1996) & (test1.index <2018)]
            test1[y] = test1["gdp2_bindar_bj"]*test1["gdp2_bindad_bj"]/100          
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)   
            print(Eqid ," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )
            
    if Eqid == "Eq7": #建立定义模型
        y = "gdp2n_bj"
        x_list =["gdp2r_bj","gdp2d_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp2n_bj","gdp2r_bj","gdp2d_bj"]]
            #test1 = test1[(test1.index>1996) & (test1.index <2018)]
            test1[y] = test1["gdp2r_bj"]*test1["gdp2d_bj"]/100        
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)   
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq8": #建立定义模型
        y = "gdp2r_bj2"
        x_list =["gdp2_indar_bj","gdp2_bindar_bj","gdp2r_erro_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp2r_bj2","gdp2_indar_bj","gdp2_bindar_bj","gdp2r_erro_bj"]]
            #test1 = test1[(test1.index>1996) & (test1.index <2018)]
            test1[y] = test1["gdp2_indar_bj"] + test1["gdp2_bindar_bj"] + test1["gdp2r_erro_bj"]       
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)   
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 

    if Eqid == "Eq9":       
        y = "gdp2r_bj1"
        x_list = ["gdp_tecn_bj","gdp2d_bj","d_gdp2r_bj_1"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp2r_bj1","gdp_tecn_bj","gdp2d_bj","d_gdp2r_bj_1"]]
            test1["gdp2r_bj1_s1"] = test1["gdp2r_bj1"].shift(1)
            test1["gdp_tecn_gdp2d"] = test1["gdp_tecn_bj"]/test1["gdp2d_bj"]
            test1 = test1[(test1.index>2003) & (test1.index <2019)]
            results = sm.ols(formula = "gdp2r_bj1 ~ gdp_tecn_gdp2d  + gdp2r_bj1_s1 + d_gdp2r_bj_1",data = test1 ).fit()            
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 
            
    if Eqid == "Eq10": #建立定义模型
        y = "gdp2r_bj"
        x_list =["gdp2r_bj1","gdp2r_bj2"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp2r_bj","gdp2r_bj1","gdp2r_bj2"]]
            #test1 = test1[(test1.index>1996) & (test1.index <2018)]
            test1[y] = (test1["gdp2r_bj1"] + test1["gdp2r_bj2"])/2      
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)   
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )

    if Eqid == "Eq11": #建立定义模型
        y = "gdp3n_bj"
        x_list =["gdp3r_bj","gdp3d_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp3n_bj","gdp3r_bj","gdp3d_bj"]]
            #test1 = test1[(test1.index>1996) & (test1.index <2018)]
            test1[y] = (test1["gdp3r_bj"] * test1["gdp3d_bj"])/100     
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)   
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] ) 

    if Eqid == "Eq12":       
        y = "gdp2_indar_bj"
        x_list = ["consr_bj","investr_bj","exusd_bj","exrate_cn","d_gdp2_indar_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp2_indar_bj","consr_bj","investr_bj","exusd_bj","exrate_cn","d_gdp2_indar_bj"]]
            test1 = test1[(test1.index>1995) & (test1.index <2019)]
            results = sm.ols(formula = "np.log(gdp2_indar_bj) ~ np.log(consr_bj)  + np.log(investr_bj) + np.log(exusd_bj * exrate_cn) + d_gdp2_indar_bj",data = test1 ).fit()           
            test1[y] = results.predict()              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )   
            
    if Eqid == "Eq13": 
        y = "gdp2_bindar_bj"
        x_list = ["investr_bj"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp2_bindar_bj","investr_bj"]]
            test1 = test1[(test1.index>1996) & (test1.index <2019)]
            ar = (0,1) 
            ma = 0
            results = sm.ols(formula = "np.log(gdp2_indar_bj) ~ np.log(consr_bj)  + np.log(investr_bj) + np.log(exusd_bj * exrate_cn) + d_gdp2_indar_bj",data = test1 ).fit()           
            test1[y] = np.exp(results.predict())[2:]  # 和AR系数有关             
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )   
            
    if Eqid == "Eq14":       
        y = "gdp3r_bj"
        x_list = ["gdp2r_bj","exusd_bj","exrate_cn"]        
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):           
            test1 = df_ev[["gdp3r_bj","gdp2r_bj","exusd_bj","exrate_cn"]]
            test1["exusexrte"] = test1["exusd_bj"]*test1["exrate_cn"]
            test1["gdp3r_bj_s1"] = test1["gdp3r_bj"].shift(1)           
            test1 = test1[(test1.index>1996) & (test1.index <2019)]
            results = sm.ols(formula = "np.log(gdp3r_bj) ~ np.log(gdp2r_bj)  + np.log(exusexrte) +  np.log(gdp3r_bj_s1)",data = test1 ).fit()
            test1[y] = np.exp(results.predict())              
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)            
            df_ev[y] = preds[y]
            exogs.remove(y)
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid , y,"还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )  
#------------------------------------还未完成 --------------------------            
    if Eqid == "Eq15": #建立定义模型
        y = "gdp3n_bj"
        x_list =["gdp3r_bj","gdp3d_bj"]
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0): 
       # try :                
            test1 = df_ev[["gdp3n_bj","gdp3r_bj","gdp3d_bj"]]
            #test1 = test1[(test1.index>1996) & (test1.index <2018)]
            test1[y] = (test1["gdp3r_bj"] * test1["gdp3d_bj"])/100     
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)              
            df_ev[y] = preds[y]                 
            exogs.remove(y)   
            print(Eqid ,y," ----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )   
                  
        #最终更新所有数据





## 111定义方程

## 173行为方程

## 3个模拟额外方程

In [ ]:

':eq01
'
':eq02
'
':eq03